In [54]:
%%html

<!----------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------------
IF YOU CAN SEE THIS CELL, CLICK IN IT, PRESS CTRL+ENTER, THEN REFRESH THE PAGE TO START THE TOOL
------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------->

<style>
/*these styles make the notebook look more like an application*/
#site {height: 100% !important;}
#notebook-container {width: 90% !important;}
h1, h2, p {margin: 0px !important;}
.anchor-link {display: none !important;}
.cell {padding: 0px !important;}
.output_subarea {padding: 2px !important;}
.output_javascript {padding: 0px !important;}
.output_html {padding: 0px !important;}
.text_cell_render {padding: 2px 0px !important; margin-top: 5px !important; border: 0 !important;}
h1, h2 {margin-top: 15px !important;}
h2 {border-bottom: thin solid !important;}
ul, .widget-button {margin: 0px !important;}
.run_this_cell {padding-top: 0px !important; padding-top: 0px !important;}

/*these styles are specific to this page*/
.fit-label {text-align: center !important; font-size: .6vw !important;}
::-webkit-clear-button {font-size: 12px !important;}
::-webkit-inner-spin-button {display: none !important;}
::-webkit-calendar-picker-indicator {font-size: 12px !important;}
input[type="date"] {font-size: 12px !important;}
.widget-toggle-button {height: 40px !important; line-height: normal !important;}
.widget-toggle-button.mod-active {background-color: rgb(249, 99, 2) !important; font-weight: bold !important;}
option {font-family: monospace !important;}
</style>

<script>
    code_show = true; 
    function code_toggle() {
        if (code_show) {
        $('div.input, bdi, #header, .ctb_hideshow').hide();
        } else {
        $('div.input, bdi, #header, .ctb_hideshow').show();
        }
        code_show = !code_show
    }
    
    function getCellsByTag(tag) {
        return Jupyter.notebook.get_cells().filter(function(c) {return (c.metadata.tags || []).includes(tag)})
    }

    function execCellsByTag(tag) {
        getCellsByTag(tag).forEach(c => c.execute())
    }
    
    function clearCellsByTag(tag) {
        getCellsByTag(tag).forEach(c => c.clear_output())
    }
    
    function showMarkdown() { //show markdown cells above lowest output of code cell - avoid having instructions on screen with no options
        var max_i = Math.max.apply(null, Jupyter.notebook.get_cells().map((c, i) => ((c.output_area || {})['outputs'] || []).length >= 1 ? i : 0));
        Jupyter.notebook.get_cells().filter((c, i) => c.cell_type == 'markdown' & i <= max_i).map(c => $(c.inner_cell).find('.text_cell_render').show())
    }
    
    function clearAllCells() {
        Jupyter.notebook.clear_cells_outputs(Array.from(Array(Jupyter.notebook.get_cells().length).keys().slice(1)))
    }
    
    function interruptCell() {
        Jupyter.notebook.kernel.interrupt()
    }

    $(document).ready(function() {
        code_toggle(); //when page is ready, hide code
        $('.text_cell_render').hide(); //hide markdown
        Jupyter.notebook.clear_cells_outputs(Array.from(Array(Jupyter.notebook.get_cells().length).keys()).slice(1)); //clear all output except this cell
    });
    
    init = true;
    Jupyter.notebook.events.on('kernel_ready.Kernel', () => {
        if (init) {
            execCellsByTag('imports');
            init = false;
        }
    });
    
    Jupyter.notebook.events.on('finished_iopub.Kernel', () => 
        showMarkdown()
    );
</script>
<a href="javascript:code_toggle()">Click here to show/hide the Python code used to build the tool</a>

In [ ]:
from google.cloud import bigquery
import ipywidgets as widgets
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from IPython.display import HTML, Javascript, display, clear_output

widgets.Widget.close_all() #notebook is doing something like looking for widget state to restore, which can overwrite newly created widgets depending on when it completes (??) this seems to prevent that by removing previous state

import sys
import datetime
import re
from dateutil.relativedelta import relativedelta
import os

#this code hides long confusing error messages from the user - run the last commented line to restore default behavior
ipython = get_ipython()
if not hasattr(ipython, 'orig_showtraceback'):
    ipython.orig_showtraceback = ipython._showtraceback
ipython._showtraceback = lambda exception_type, exception, traceback: print(exception, file=sys.stderr)
ipython._showtraceback = ipython.orig_showtraceback


client = bigquery.Client(project='analytics-supplychain-thd')#May have to specify project here

# Supply Chain Data Science Forecasting Tool

In [2]:

#diplay formatted widget with desired description
def display_with_description(widget, description, width=.15):
    width_string = str(int(width * 100))+'%'
    other_width_string = str(int((1-width) * 100))+'%'
    label = widgets.Label(value=description, layout=widgets.Layout(width=width_string))
    widget.layout.width = other_width_string
    display(widgets.HBox([label, widget]))

#string replace queries with inputs
def genericReplace(query,replaceDict):
    for i,j in replaceDict.items():
        query = query.replace(i,str(j))
    return query

global outputCode
outputCode = {'createForecast' : '', 'insertForecast' : '', 'ensemble': '', 'adjustments' : ''}

#exec query and append to dictionary to be exported

######altSQL = indicator that code is run for current_date only
def exec_sql(sql, stringReplaceDict, codeBlock, altSQL=None):
    newReplaceDict = stringReplaceDict.copy()
    sql = genericReplace(sql, newReplaceDict)
    if altSQL == None:
        outputCode[codeBlock] += sql
    else:#if SQL run will be changed for BQA export
        altSQL = genericReplace(altSQL, newReplaceDict)
        outputCode[codeBlock] += altSQL
    client.query(sql).result()

#concatenate inputs columns into single column
def make_hierarchy(l):
    return ''', '|', '''.join(['CAST({} AS STRING)'.format(c) for c in l])

display(Javascript("clearCellsByTag('imports')"))

display(Javascript("execCellsByTag('makeUI')"))

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [3]:
  
presets_input = widgets.Dropdown(options=[" ","Weekly Lane Truckloads", "Weekly Store Sku Sales"])

def restore_version(self):
    if presets_input.value == "Weekly Lane Truckloads":
        #"Weekly Lane Truckloads", "Weekly Store Sku Sales"
        timePeriod_input.value = 'FSCL_WK_END_DT'
        hierarchy_input.value = 'origin, destin'
        historicValue_input.value = 'loads'
        snapshotPeriods_input.value = '3'
        dataset_input.value = 'analytics-supplychain-thd.one_forecast_truck'
        outputTable_input.value = 'one_forecast_truck'
        forecastPeriodStart_input.value = datetime.date(2019, 1, 1)
        forecastPeriodEnd_input.value = datetime.date(2019, 12, 31)
        bucketPath_input.value = 'gs://sca-udf/reinforcejs-master/lib'
        rawData_input.value = '`analytics-supplychain-thd.one_forecast_truck.inputs`'
    elif presets_input.value == "Weekly Store Sku Sales":
        timePeriod_input.value = 'FSCL_WK_END_DT'
        hierarchy_input.value = 'str_nbr, sku_nbr'
        historicValue_input.value = 'gross_unt_sls'
        snapshotPeriods_input.value = '1'
        dataset_input.value = 'analytics-supplychain-thd.one_forecast'
        forecastPeriodStart_input.value = datetime.date(2019, 2, 4)
        forecastPeriodEnd_input.value = datetime.date(2020, 2, 2)
        bucketPath_input.value = 'gs://sca-udf/reinforcejs-master/lib'
        rawData_input.value = '`analytics-supplychain-thd.one_forecast.str_sku_weekly_D28_C10`'

presets_input.observe(restore_version)


display_with_description(presets_input, 'Choose from preset options:')

## Forecast Creation Input:
Create up to three forecasts based on time series input

In [4]:
#timePeriod_input = widgets.Text(placeholder='Name of date column in input table', save_id='timePeriod_input')
timePeriod_input = widgets.Dropdown(options=[" ","CAL_DT", "FSCL_WK_END_DT"], save_id = 'timePeriod_input')
hierarchy_input = widgets.Text(placeholder='List of column names to aggregate forecast at (ex: dept, class, subclass or origin, destination)', save_id='hierarchy_input')
historicValue_input = widgets.Text(placeholder='Column name of target value to forecast', save_id='historicValue_input')
snapshotPeriods_input = widgets.Text(placeholder='Number of periods to forecast from each forecast snapshot', save_id='snapshotPeriods_input')
dataset_input = widgets.Text(placeholder='Dataset to output forecast to (ex: analytics-supplychain-thd.one_forecast)', save_id='dataset_input')
outputTable_input = widgets.Text(placeholder='Table to ouput forecast to (ex: one_forecast)', save_id='dataset_input')
forecastPeriodStart_input = widgets.DatePicker(save_id='forecastPeriodStart_input')
forecastPeriodEnd_input = widgets.DatePicker(save_id='forecastPeriodEnd_input')
rawData_input = widgets.Text(placeholder='Input Data Table (ex: analytics-supplychain-thd.one_forecast.inputs)', save_id='rawData_input')
frequency_input = widgets.Dropdown(options=["Week","Day"])
run_btn = widgets.Button(description='Create Forecast', icon='upload', layout=widgets.Layout(width='100%'), button_style='success')

clear_btn = widgets.Button(description='Clear Output', icon='trash', layout=widgets.Layout(width='100%'), button_style='primary')
cancel_btn = widgets.Button(description='Cancel Run', icon='ban', layout=widgets.Layout(width='100%'), button_style='danger').add_class('thd_cancel_button')

out = widgets.Output(layout={'border': '1px solid black'})
tempOut = widgets.Output(layout={'border': '1px solid black'})

round_box = widgets.Checkbox(value=False,description='Round Output Values',disabled=False)
negative_box = widgets.Checkbox(value=False,description='Disallow Negative Output Values',disabled=False)


#create forecast with current inputs
def createForecast(self):
    try:
        outputCode['createForecast'] = '/******************************** Create Forecast ********************************/\n'
        timePeriod = timePeriod_input.value#one column date, can be any frequency daily and up
        global hierarchyColumns
        hierarchyColumns = hierarchy_input.value.split(',')#save column names to deaggregate later
        for i in range(len(hierarchyColumns)):
            hierarchyColumns[i] = hierarchyColumns[i].strip()
        hierarchy = hierarchyColumns.copy()
        for i in range(len(hierarchy)):
            hierarchy[i] = 'cast(' + hierarchy[i] + ' as string)'
        forecastValue = historicValue_input.value#one int or float column, target value to forecast
        snapshotPeriods = snapshotPeriods_input.value#number of weeks to forecast
        dataset = dataset_input.value.replace('`','')#dataset to store forecast in
        forecastPeriodStart = forecastPeriodStart_input.value
        forecastPeriodEnd = forecastPeriodEnd_input.value
        transform1 = ''
        transform2 = ''
        if round_box.value:
            if negative_box.value:
                transform1 = 'round(greatest('
                transform2 = ',0))'
            else:
                transform1 = 'round('
                transform2 = ')'
        elif negative_box.value:
            transform1 = 'greatest('
            transform2 = ',0)'
        frequency = frequency_input.value#day, week or month
        JSCloudPath = bucketPath_input.value
        inputTable = rawData_input.value.replace('`','')
        hierarchyString = ''', '|', '''.join(hierarchy)
        outputTable = outputTable_input.value.replace('`', '').split('.')[-1]

        global replaceDict
        replaceDict = {'genericTimePeriod':timePeriod, 'genericHierarchy':hierarchyString, 'genericForecastValue':forecastValue
                      ,'snapshotPeriods': snapshotPeriods, 'genericDataset': dataset#,'startYear' : startYear, 'endYear' : endYear
                      ,'forecastPeriodStart': forecastPeriodStart, 'forecastPeriodEnd': forecastPeriodEnd, 'genericFrequency':frequency
                      , 'genericInputTable': inputTable, 'plusOneSnapshotPeriods': str(int(snapshotPeriods) + 1), 'genericOutputTable' : outputTable}


        if frequency.lower() == 'day':
            replaceDict['genericSeasonality'] = "[STRUCT('DAILY' AS TYPE, DAILY_SEASON AS SEASON, DAILY_HOLIDAY_FLG AS HOLIDAY_FLG), STRUCT('WEEKLY' AS TYPE, WEEKLY_SEASON AS SEASON, WEEKLY_HOLIDAY_FLG AS HOLIDAY_FLG)]"
            replaceDict['genericNumPeriods'] = 364 #currently this must be even
            replaceDict['genericCalendarColumn'] = 'CAL_DT'
            seasonal_config = [["'DAILY'", 1], ["'WEEKLY'", 5]]
        elif frequency.lower() == 'week':
            replaceDict['genericSeasonality'] = "[STRUCT('WEEKLY' AS TYPE, WEEKLY_SEASON AS SEASON, WEEKLY_HOLIDAY_FLG AS HOLIDAY_FLG)]"
            replaceDict['genericNumPeriods'] = 52
            replaceDict['genericCalendarColumn'] = 'FSCL_WK_END_DT'
            seasonal_config = [["'WEEKLY'", 5]]
        else:
            replaceDict['genericSeasonality'] = "[STRUCT('MONTHLY' AS TYPE, MONTHLY_SEASON AS SEASON, FALSE AS HOLIDAY_FLG)]"
            replaceDict['genericNumPeriods'] = 12
            replaceDict['genericCalendarColumn'] = 'FSCL_PRD_END_DT'
            seasonal_config = [["'MONTHLY'", 3]]
        replaceDict['genericSeasonalLags'] = max(x[1] for x in seasonal_config)
        replaceDict['genericSeasonalUnion'] = ' UNION ALL '.join(['SELECT {} AS TYPE, {} AS LAGS'.format(x[0], x[1]) for x in seasonal_config])
        replaceDict['genericTableOptionList'] = 'OPTIONS(EXPIRATION_TIMESTAMP=TIMESTAMP_ADD(CURRENT_TIMESTAMP, INTERVAL 1 DAY))'


        if '' in replaceDict.values():
            with out:
                print(datetime.datetime.now().strftime('%H:%M:%S'), 'Error: missing input detected')
            return
        replaceDict['forecastTransform1'] = transform1
        replaceDict['forecastTransform2'] = transform2
        with out:
            print(datetime.datetime.now().strftime('%H:%M:%S'), 'Creating forecast...')
            print(genericReplace('Forecast(s) will be stored in `genericDataset.genericOutputTable`...',replaceDict))

        replaceDict['genericTableOptionList'] = 'OPTIONS(EXPIRATION_TIMESTAMP=TIMESTAMP_ADD(CURRENT_TIMESTAMP, INTERVAL 1 DAY))'
        exec_sql(
           'create or replace table `genericDataset.raw_forecast_data` genericTableOptionList as ( select * from `'
             + inputTable
             + '`);',
        replaceDict, 'createForecast')


        with tempOut:
            %store replaceDict

        display(Javascript("execCellsByTag('Forecast1')"))
    except:
        with out:
            print(datetime.datetime.now().strftime('%H:%M:%S'), 'Error: Invalid input data detected. Please verify query and try again.')
    
    
#clear all outputs
def clearOutput(self):
    out.clear_output()
    insertOut.clear_output()
    selectionOut.clear_output()
    adjustOut.clear_output()
    exportOut.clear_output()
    


display_with_description(timePeriod_input, 'Input Table Date Column:')
display_with_description(hierarchy_input, 'Forecast Value Hierarchy:')
display_with_description(historicValue_input, 'Forecast Value:')
display_with_description(snapshotPeriods_input, 'Forecast Periods:')
display_with_description(dataset_input, 'Output Dataset:')
display_with_description(outputTable_input, 'Final Output Table:')
display_with_description(forecastPeriodStart_input, 'Forecast Snapshot Start:')
display_with_description(forecastPeriodEnd_input, 'Forecast Snapshot End:')
display_with_description(frequency_input, 'Forecast Frequency:')
display_with_description(rawData_input, 'Input Data Table:')
display(widgets.HBox([round_box, negative_box]))

display(run_btn)
display(widgets.HBox([clear_btn, cancel_btn]))
display(out)

run_btn.on_click(createForecast)
clear_btn.on_click(clearOutput)
display(Javascript('''
$('.thd_cancel_button').click(function() {Jupyter.notebook.kernel.interrupt()
    }
)
'''))


Button(button_style='success', description='Create Forecast', icon='upload', layout=Layout(width='100%'), styl…

Output(layout=Layout(border='1px solid black'))

<IPython.core.display.Javascript object>

## External Forecast Input (Optional):
Add an external forecast to the one forecast table to be considered during the ensemble approach

In [ ]:
testString = "`analytics-supplychain-thd.one_forecast_daily_test.one_forecast_test"
test = testString.replace('`', '').split('.')[-1]
print(test)

In [5]:
def extractColumnNames(client, table):
    project = table.split('.')[0]
    dataset_id = table.split('.')[1]
    table_id = table.split('.')[2]

    dataset_ref = client.dataset(dataset_id, project=project)
    table_ref = dataset_ref.table(table_id)
    table = client.get_table(table_ref)  # API Request
    
    snapshotTime = ''
    fcstTime = ''
    forecastValue = ''
    
    for field in table.schema:
        if re.search('snpsht_.+',field.name.lower()):
            snapshotTime = field.name
        if re.search('fcst_.+',field.name.lower()):
            fcstTime = field.name
        if re.search('forecast4_.+',field.name.lower()):
            forecastValue = field.name
    return([snapshotTime, fcstTime, forecastValue])



#Input outside forecast option
fcstHierarchy_input = widgets.Text(placeholder='List of column names to aggregate forecast at (ex: dept, class, subclass or origin, destination)', save_id='fcstHierarchy_input')
fcstInsertTable_input = widgets.Text(placeholder='Table to input forecast to (ex: analytics-supplychain-thd.one_forecast.one_forecast)', save_id='fcstDataset_input')
snapshotTimePeriod_input = widgets.Text(placeholder='Name of create date column in input query', save_id='snapshotTimePeriod_input')
fcstTimePeriod_input = widgets.Text(placeholder='Name of forecast date column in input query', save_id='fcstTimePeriod_input')
forecastValue_input = widgets.Text(placeholder='Target value from forecast input', save_id='forecastValue_input')
newForecast_input = widgets.Textarea(placeholder='External Forecast Query (Must have columns representing Forecast Create Time, Forecast Time and Forecast Value)', save_id='newForecast_input', layout = widgets.Layout(height = '120px'))
upload_btn = widgets.Button(description='Insert Additional Forecast', icon='upload', layout=widgets.Layout(width='100%'), button_style='success')
insertOut = widgets.Output(layout={'border': '1px solid black'})
            

#joins input forecast into one_forecast table as "forecast4"
def insertForecast(self):
    outputCode['insertForecast'] = '/******************************** Insert Forecast ********************************/\n'
    hierarchyString = ''', '|', '''.join(fcstHierarchy_input.value.split(','))
    fcstInsertTable = fcstInsertTable_input.value.replace('`', '')
    try:
        insertReplaceDict = {'snpsht_genericTimePeriod':snapshotTimePeriod_input.value, 'fcst_genericTimePeriod':fcstTimePeriod_input.value
                     ,'genericHierarchy':hierarchyString, 'newForecastValue':forecastValue_input.value
                      ,'genericInsertTable': fcstInsertTable, 'genericForecastValue':extractColumnNames(client, fcstInsertTable)[2]
                    ,'genericDataset': '.'.join(fcstInsertTable.split('.', 2)[:2])}
        with insertOut:
            print(datetime.datetime.now().strftime('%H:%M:%S'), genericReplace('Adding input forecast to genericInsertTable',insertReplaceDict))
        exec_sql(
            ''' CREATE OR REPLACE TABLE `genericDataset.externalForecast` as
                (
                with tbl1 as (
                %s
                )
                select snpsht_genericTimePeriod, fcst_genericTimePeriod, newForecastValue, concat(genericHierarchy) as aggLevel
                from tbl1
                );
                ''' % (newForecast_input.value)
        ,insertReplaceDict, 'insertForecast')
                
        exec_sql(   
                '''update `genericInsertTable` a
                set a.genericForecastValue = b.newForecastValue
                FROM `genericDataset.externalForecast` b
                where a.%s = b.snpsht_genericTimePeriod
                and a.%s = b.fcst_genericTimePeriod
                and a.aggLevel = b.aggLevel;
            ''' % (extractColumnNames(client, fcstInsertTable)[0], extractColumnNames(client, fcstInsertTable)[1])
            ,insertReplaceDict, 'insertForecast')
        with insertOut:
            print(datetime.datetime.now().strftime('%H:%M:%S'), genericReplace('Forecast successfully added to genericInsertTable',insertReplaceDict))
    except:
        with insertOut:
            print(datetime.datetime.now().strftime('%H:%M:%S'), 'Error: Invalid inputs detected. Please verify query and try again.')
        return

display_with_description(fcstHierarchy_input, 'Forecast Value Hierarchy:')
display_with_description(fcstInsertTable_input, 'One Forecast Engine Table:')
display_with_description(snapshotTimePeriod_input, 'Forecast Created Time Period:')
display_with_description(fcstTimePeriod_input, 'Forecast Time Period:')
display_with_description(forecastValue_input, 'Forecast Value:')

display_with_description(newForecast_input, 'Forecast Input Query:')
display(upload_btn)
display(insertOut)

upload_btn.on_click(insertForecast)

Button(button_style='success', description='Insert Additional Forecast', icon='upload', layout=Layout(width='1…

Output(layout=Layout(border='1px solid black'))

## Ensemble:
Combine previously made forecasts into the best possible forecast

In [6]:
bucketPath_input = widgets.Text(placeholder='Path to Cloud Storage Bucket (Only Required for Reinforcement Learning Selection Logic - default location: gs://sca-udf/reinforcejs-master/lib)', save_id='bucketPath_input')
selection_input = widgets.Dropdown(options=["","Reinforcement Learning Selection","Rules Based Selection"], save_id ='selection_input')

forecast4_drop = widgets.Dropdown(options = ["No", "Yes"], save_id = 'forecast4_input')

display_with_description(selection_input, 'Forecast Selection Logic:')

selectionOut = widgets.Output(layout={'border': '1px solid black'})

#runs selected ensemble logic cell
def runEnsemble(self):
    try:
        replaceDict
    except:
        %store -r replaceDict
    global ensembleReplaceDict
    ensembleReplaceDict = replaceDict.copy()
    if forecast4_drop.value == 'Yes':
        ensembleReplaceDict['forecast3'] = 'forecast4'
    outputCode['ensemble'] = '/******************************** Selection Logic ********************************/\n'
    ensembleReplaceDict['JSCloudPath'] = bucketPath_input.value
    if (selection_input.value == '' or (selection_input.value == "Reinforcement Learning Selection" and bucketPath_input.value == '')):
        with selectionOut:
            print(datetime.datetime.now().strftime('%H:%M:%S'), 'Error: Invalid input data query detected. Please verify query and try again.')
        return
    with selectionOut:
        print(datetime.datetime.now().strftime('%H:%M:%S'), 'Creating Selection Logic Input Tables...')
    display(Javascript("execCellsByTag('SelectionInput')"))
    with selectionOut:
        print(datetime.datetime.now().strftime('%H:%M:%S'), genericReplace('Running chosen selection logic, results will be stored in `genericDataset.genericOutputTable`...',replaceDict))
    if selection_input.value == "Reinforcement Learning Selection":
        display(Javascript("execCellsByTag('RLSelection')"))
    elif selection_input.value == "Rules Based Selection":
        display(Javascript("execCellsByTag('RulesSelection')"))
    elif selection_input.value == "Logistic Regression Selection":
        display(Javascript("execCellsByTag('LogRegSelection')"))

        
display_with_description(bucketPath_input, 'Cloud Storage Bucket Path:')
display_with_description(forecast4_drop, 'Include External Forecast Input?')

ensemble_btn = widgets.Button(description='Run Forecast Ensemble', icon='upload', layout=widgets.Layout(width='100%'), button_style='success')
ensemble_btn.on_click(runEnsemble)
display(ensemble_btn)

display(selectionOut)

Button(button_style='success', description='Run Forecast Ensemble', icon='upload', layout=Layout(width='100%')…

Output(layout=Layout(border='1px solid black'))

## Forecast Analysis (Optional):
Plot newly created forecasts and error

In [7]:
MAPE_btn = widgets.Button(description='Plot Forecast MAPE', icon='signal', layout=widgets.Layout(width='100%'), button_style='success')
lagMeasured_input = widgets.Text(placeholder='Forecast Lag to Plot', save_id='lagMeasured_input')
plotPeriodStart_input = widgets.DatePicker(placeholder='Plot Start Date', save_id='plotPeriodStart_input')
plotPeriodEnd_input = widgets.DatePicker(placeholder='Plot End Date', save_id='plotPeriodEnd_input')

display_with_description(lagMeasured_input, 'Forecast Lag to Plot:')
display_with_description(plotPeriodStart_input, 'Plot Period Start:')
display_with_description(plotPeriodEnd_input, 'Plot Period End:')
plotPeriodStart_input.value = datetime.date.today() - relativedelta(years = 1)
plotPeriodEnd_input.value = datetime.date.today()

display(MAPE_btn)

#plots designated time period vs overall forecast MAPE chart
def graphMAPE(self):
    try:
        try:
            replaceDict
        except:
            %store -r replaceDict
        global plotReplaceDict
        plotReplaceDict = replaceDict.copy()
        plotReplaceDict['forecastLag'] = int(lagMeasured_input.value) - 1
        plotReplaceDict['plotPeriodStart'] = str(plotPeriodStart_input.value)
        plotReplaceDict['plotPeriodEnd'] = str(plotPeriodEnd_input.value)
        display(Javascript("clearCellsByTag('graphMAPE')"))
        display_with_description(lagMeasured_input, 'Forecast Lag to Plot:')
        display_with_description(plotPeriodStart_input, 'Plot Period Start:')
        display_with_description(plotPeriodEnd_input, 'Plot Period End:')
        display(MAPE_btn)
        df = pd.read_gbq(genericReplace('''SELECT snpsht_genericTimePeriod as snpsht_time, fcst_genericTimePeriod as fcst_time
        , safe_divide(sum(abs(forecast1_genericForecastValue - actual_genericForecastValue)),sum(actual_genericForecastValue)) forecast1_MAPE
        , safe_divide(sum(abs(forecast2_genericForecastValue - actual_genericForecastValue)),sum(actual_genericForecastValue)) forecast2_MAPE
        , safe_divide(sum(abs(forecast3_genericForecastValue - actual_genericForecastValue)),sum(actual_genericForecastValue)) forecast3_MAPE
        , safe_divide(sum(abs(forecast4_genericForecastValue - actual_genericForecastValue)),sum(actual_genericForecastValue)) forecast4_MAPE
        , safe_divide(sum(abs(one_forecast_genericForecastValue - actual_genericForecastValue)),sum(actual_genericForecastValue)) one_forecast_MAPE
        FROM `genericDataset.genericOutputTable`
        where DATE_DIFF(fcst_genericTimePeriod, snpsht_genericTimePeriod,genericFrequency) = forecastLag
        and actual_genericForecastValue is not null
        --and actual_genericForecastValue > 0
        and fcst_genericTimePeriod between 'plotPeriodStart' and 'plotPeriodEnd'
        group by 1,2
        order by 1,2''',plotReplaceDict)
            , dialect='standard'
            ,configuration = {'query': {'useQueryCache': False}})

        plt.axes(xlabel = 'Date', ylabel = 'MAPE', title = 'Forecast MAPE')
        forecast1, = plt.plot(df.fcst_time, df.forecast1_MAPE,label = 'Forecast 1')
        forecast2, = plt.plot(df.fcst_time, df.forecast2_MAPE,label = 'Forecast 2')
        forecast3, = plt.plot(df.fcst_time, df.forecast3_MAPE,label = 'Forecast 3')
        forecast4, = plt.plot(df.fcst_time, df.forecast4_MAPE,label = 'Forecast 4')
        one_forecast, = plt.plot(df.fcst_time, df.one_forecast_MAPE,label = 'One Forecast')
        plt.legend(handles=[forecast1, forecast2, forecast3, forecast4, one_forecast])
        matplotlib.rcParams['figure.figsize'] = [24,9]
        matplotlib.rcParams['figure.dpi'] = 80

        plt.show()
    except:
        print(datetime.datetime.now().strftime('%H:%M:%S'), 'Error: Invalid input data query detected. Please verify query and try again. Note that a forecast must have been created in the current session to be plotted.')
    


MAPE_btn.on_click(graphMAPE)

Button(button_style='success', description='Plot Forecast MAPE', icon='signal', layout=Layout(width='100%'), s…

In [8]:
graph_btn = widgets.Button(description='Plot Forecasts', icon='signal', layout=widgets.Layout(width='100%'), button_style='success')
display(graph_btn)

#plot forecasts vs actuals
def graphForecasts(self):
    try:
        try:
            replaceDict
        except:
            %store -r replaceDict
        global plotReplaceDict
        plotReplaceDict = replaceDict.copy()
        plotReplaceDict['forecastLag'] = int(lagMeasured_input.value) - 1
        plotReplaceDict['plotPeriodStart'] = str(plotPeriodStart_input.value)
        plotReplaceDict['plotPeriodEnd'] = str(plotPeriodEnd_input.value)
        display(Javascript("clearCellsByTag('graphForecast')"))
        display(graph_btn)
        df = pd.read_gbq(genericReplace('''SELECT snpsht_genericTimePeriod as snpsht_time, fcst_genericTimePeriod as fcst_time
        , sum(forecast1_genericForecastValue) as forecast1_value
        , sum(forecast2_genericForecastValue) as forecast2_value
        , sum(forecast3_genericForecastValue) as forecast3_value
        , sum(forecast4_genericForecastValue) as forecast4_value
        , sum(one_forecast_genericForecastValue) as one_forecast_value
        , sum(actual_genericForecastValue) as actual_value
        FROM `genericDataset.genericOutputTable`
        where DATE_DIFF(fcst_genericTimePeriod, snpsht_genericTimePeriod,genericFrequency) = forecastLag
        and actual_genericForecastValue is not null
        and fcst_genericTimePeriod between 'plotPeriodStart' and 'plotPeriodEnd'
        group by 1,2
        order by 1,2''',plotReplaceDict)
            , dialect='standard'
            ,configuration = {'query': {'useQueryCache': False}})

        plt.axes(xlabel = 'Date', ylabel = genericReplace('genericForecastValue', replaceDict), title = 'Forecast vs Actuals')
        forecast1, = plt.plot(df.fcst_time, df.forecast1_value,label = 'Forecast 1')
        forecast2, = plt.plot(df.fcst_time, df.forecast2_value,label = 'Forecast 2')
        forecast3, = plt.plot(df.fcst_time, df.forecast3_value,label = 'Forecast 3')
        forecast4, = plt.plot(df.fcst_time, df.forecast4_value,label = 'Forecast 4')
        one_forecast, = plt.plot(df.fcst_time, df.one_forecast_value,label = 'one_forecast')
        actuals, = plt.plot(df.fcst_time, df.actual_value,label = 'Actuals')
        plt.legend(handles=[forecast1, forecast2, forecast3, forecast4, one_forecast, actuals])

        matplotlib.rcParams['figure.figsize'] = [24,9]
        matplotlib.rcParams['figure.dpi'] = 80
        plt.show()
    except:
        print(datetime.datetime.now().strftime('%H:%M:%S'), 'Error: Invalid input data query detected. Please verify query and try again. Note that a forecast must have been created in the current session to be plotted.')
    
graph_btn.on_click(graphForecasts)

Button(button_style='success', description='Plot Forecasts', icon='signal', layout=Layout(width='100%'), style…

## Forecast Adjustments (Optional):
Make adjustments to the recently created forecast

In [9]:
forecastAdjustment = widgets.Dropdown(options=["", "Reactive Adjustments - Adjust to consistent over/underforecasting over last 3 periods"
                                               ,"Forecast Increase - Across the board increase in forecast values by x%"
                                              ,"Forecast Decrease - Across the board decrease in forecast values by x%"])
adjustPeriods_input = widgets.Text(placeholder='Number of periods to adjust off of (n)', save_id='adjustPeriods_input')
adjustPercent_input = widgets.Text(placeholder='% forecast adjustment (x)', save_id='adjustPercent_input')
adjust_btn = widgets.Button(description='Adjust Forecast', icon='gear', layout=widgets.Layout(width='100%'), button_style='success')
adjustOut = widgets.Output(layout={'border': '1px solid black'})


display_with_description(forecastAdjustment, "Forecast Adjustment Options")
#display_with_description(adjustPeriods_input, "Adjustment Periods")
display_with_description(adjustPercent_input, "Adjustment Percentage")
display(adjust_btn)
display(adjustOut)
    
#adjust forecasts based on selected inputs
def adjustForecast(self):
    outputCode['adjustments'] = '/******************************** Forecast Adjustments ********************************/\n'
    try:
        try:
            replaceDict
        except:
            %store -r replaceDict
        with adjustOut:
            print(datetime.datetime.now().strftime('%H:%M:%S'), genericReplace('Forecasts stored in `genericDataset.genericOutputTable` will be adjusted accordingly...', replaceDict))
        replaceDict['adjustPercent'] = adjustPercent_input.value
        replaceDict['adjustPeriods'] = adjustPeriods_input.value
        if forecastAdjustment.value == "Reactive Adjustments - Adjust to consistent over/underforecasting over last 3 periods":
            exec_sql(
            '''
            create temp function MAPE_adjust(forecasts ARRAY<STRUCT<fcst_genericTimePeriod date, forecast1_genericForecastValue float64, forecast2_genericForecastValue float64
                    , forecast3_genericForecastValue float64, forecast1_diff float64, forecast2_diff float64, forecast3_diff float64, actual_genericForecastValue float64>>)
            RETURNS ARRAY<STRUCT<fcst_genericTimePeriod date, forecast1_genericForecastValue float64, adj_forecast1_genericForecastValue float64
            , forecast2_genericForecastValue float64, adj_forecast2_genericForecastValue float64
            , forecast3_genericForecastValue float64, adj_forecast3_genericForecastValue float64, actual_genericForecastValue float64>>
            LANGUAGE js as """

              var data = forecasts.map(function(obj) {
                return Object.keys(obj).map(function(key) {
                  return obj[key]
                })
              })


              var week = []
              var unadjusted_vals = []
              var adjusted_vals = []
              var diffs = []
              var i = 0
              while(i < data.length) {
                  week.push(data[i][0])
                  unadjusted_vals.push(data[i].slice(1,4))
                  adjusted_vals.push(data[i].slice(1,4))
                  diffs.push(data[i].slice(4,7))
                  i++
              }

              var wk = 0
              var output = []
              var trendWeeks = 3
              while(wk < week.length) {
                var fcst = 0
                while (fcst < unadjusted_vals.length - 1 && wk >= trendWeeks) {
                  if ((diffs[wk-1][fcst] < 0 && diffs[wk-2][fcst] < 0 && diffs[wk-3][fcst] < 0) || (diffs[wk-1][fcst] > 0 && diffs[wk-2][fcst] > 0 && diffs[wk-3][fcst] > 0)) {
                      var avgDiff = (diffs[wk-1][fcst] + diffs[wk-2][fcst] + diffs[wk-3][fcst])/3
                      if (unadjusted_vals[wk][fcst] - avgDiff > 0) {
                          adjusted_vals[wk][fcst] = unadjusted_vals[wk][fcst] - avgDiff
                      } else {
                          adjusted_vals[wk][fcst] = 0
                      }
                  }
                  fcst++
                }
                output.push({fcst_genericTimePeriod:week[wk], adj_forecast1_genericForecastValue:Math.round(adjusted_vals[wk][0]), adj_forecast2_genericForecastValue:Math.round(adjusted_vals[wk][1])
                , adj_forecast3_genericForecastValue:Math.round(adjusted_vals[wk][2]), forecast1_genericForecastValue:unadjusted_vals[wk][0], forecast2_genericForecastValue:unadjusted_vals[wk][1]
                , forecast3_genericForecastValue:unadjusted_vals[wk][2], actual_genericForecastValue:data[wk][7]})
                wk++
              }

              return output


            """
            OPTIONS ();

            update `genericDataset.genericOutputTable` a
            set
            a.forecast1_genericForecastValue = b.adj_forecast1_genericForecastValue
            ,a.forecast2_genericForecastValue = b.adj_forecast2_genericForecastValue
            ,a.forecast3_genericForecastValue = b.adj_forecast3_genericForecastValue
            from
            (select 
            aggLevel, b.* from
            (select aggLevel,
            MAPE_adjust(ARRAY_AGG(STRUCT( fcst_genericTimePeriod , forecast1_genericForecastValue, forecast2_genericForecastValue, forecast3_genericForecastValue
            , forecast1_diff, forecast2_diff, forecast3_diff, coalesce(actual_genericForecastValue,0) as actual_genericForecastValue)
                                  order by fcst_genericTimePeriod )) as adj_forecasts
            from
            (select SNPSHT_genericTimePeriod , fcst_genericTimePeriod , aggLevel, forecast1_genericForecastValue,forecast2_genericForecastValue,forecast3_genericForecastValue, actual_genericForecastValue
            , forecast1_genericForecastValue - actual_genericForecastValue as forecast1_diff
            , forecast2_genericForecastValue - actual_genericForecastValue as forecast2_diff
            , forecast3_genericForecastValue - actual_genericForecastValue as forecast3_diff
            from `genericDataset.genericOutputTable`
            where ( SNPSHT_genericTimePeriod = fcst_genericTimePeriod
            and fcst_genericTimePeriod >= CAST(cast('forecastPeriodStart' as string) as date)
            and fcst_genericTimePeriod <= CAST(cast('forecastPeriodEnd' as string) as date))
            order by 1,2)
            group by 1) a
            ,unnest(adj_forecasts) b
            ) b
            where
            a.aggLevel = b.aggLevel
            and a.fcst_genericTimePeriod = b.fcst_genericTimePeriod
            and a.snpsht_genericTimePeriod = a.fcst_genericTimePeriod;
            ''',replaceDict, 'adjustments')
            with adjustOut:
                print(datetime.datetime.now().strftime('%H:%M:%S'), genericReplace('Forecasts in `genericDataset.genericOutputTable` adjusted based on preceding 3 genericFrequency periods.', replaceDict))
        elif forecastAdjustment.value == "Forecast Increase - Across the board increase in forecast values by x%":
            exec_sql('''update `genericDataset.genericOutputTable` a
                    set a.forecast1_genericForecastValue = forecastTransform1 b.forecast1_genericForecastValue * (1 + abs(adjustPercent/100)) forecastTransform2
                    ,a.forecast2_genericForecastValue = forecastTransform1 b.forecast2_genericForecastValue * (1 + abs(adjustPercent/100)) forecastTransform2
                    ,a.forecast3_genericForecastValue = forecastTransform1 b.forecast3_genericForecastValue * (1 + abs(adjustPercent/100)) forecastTransform2
                    ,a.forecast4_genericForecastValue = forecastTransform1 b.forecast4_genericForecastValue * (1 + abs(adjustPercent/100)) forecastTransform2
                    from `genericDataset.genericOutputTable` b
                    where a.snpsht_genericTimePeriod = b.snpsht_genericTimePeriod
                    and a.fcst_genericTimePeriod = b.fcst_genericTimePeriod
                    and a.aggLevel = b.aggLevel;
                ''',replaceDict, 'adjustments')
            with adjustOut:
                print(datetime.datetime.now().strftime('%H:%M:%S'), genericReplace('Forecasts in `genericDataset.genericOutputTable` increased by adjustPercent%.', replaceDict))
        elif forecastAdjustment.value == "Forecast Decrease - Across the board decrease in forecast values by x%":
                exec_sql('''update `genericDataset.genericOutputTable` a
                set a.forecast1_genericForecastValue = forecastTransform1 b.forecast1_genericForecastValue * (1 - abs(adjustPercent/100)) forecastTransform2
                ,a.forecast2_genericForecastValue = forecastTransform1 b.forecast2_genericForecastValue * (1 - abs(adjustPercent/100)) forecastTransform2
                ,a.forecast3_genericForecastValue = forecastTransform1 b.forecast3_genericForecastValue * (1 - abs(adjustPercent/100)) forecastTransform2
                ,a.forecast4_genericForecastValue = forecastTransform1 b.forecast4_genericForecastValue * (1 - abs(adjustPercent/100)) forecastTransform2
                from `genericDataset.genericOutputTable` b
                where a.snpsht_genericTimePeriod = b.snpsht_genericTimePeriod
                and a.fcst_genericTimePeriod = b.fcst_genericTimePeriod
                and a.aggLevel = b.aggLevel;
                ''',replaceDict, 'adjustments')
                with adjustOut:
                    print(datetime.datetime.now().strftime('%H:%M:%S'), genericReplace('Forecasts in `genericDataset.genericOutputTable` decreased by adjustPercent%.', replaceDict))
        else:
            with adjustOut:
                print(datetime.datetime.now().strftime('%H:%M:%S'), 'Error: Invalid input detected while adjusting genericOutputTable table. Please verify inputs and try again.')

    except:
        with adjustOut:
            print(datetime.datetime.now().strftime('%H:%M:%S'), 'Error: Invalid input detected while adjusting genericOutputTable table. Please verify inputs and try again.')
            
adjust_btn.on_click(adjustForecast)



Button(button_style='success', description='Adjust Forecast', icon='gear', layout=Layout(width='100%'), style=…

Output(layout=Layout(border='1px solid black'))

## Export Forecast:
Download the most recently run forecast code

In [10]:
export_btn = widgets.Button(description='Export Forecast', icon='download', layout=widgets.Layout(width='100%'), button_style='success')
exportOut = widgets.Output(layout={'border': '1px solid black'})

#export previously run code to txt file in cwd
def exportForecast(self):
    text_file = open("output.txt", "w")
    text_file.write('{createForecast} \n {insertForecast} \n {adjustments} \n {ensemble}'.format(**outputCode))
    text_file.close()
    with exportOut:
        print(datetime.datetime.now().strftime('%H:%M:%S'), 'Most recent forecast run output to the output.txt file in the directory: %s' % (os.getcwd()))

export_btn.on_click(exportForecast)
display(export_btn)
display(exportOut)



Button(button_style='success', description='Export Forecast', icon='download', layout=Layout(width='100%'), st…

Output(layout=Layout(border='1px solid black'))

In [ ]:
try:
    exec_sql(
       '''
            
            create or replace table `genericDataset.forecast_data` genericTableOptionList as (
            
            with tbl1 as (
            select a.*, coalesce(b.genericForecastValue,0) as genericForecastValue from
            (select * from
            (select distinct concat(genericHierarchy) as aggLevel from `genericDataset.raw_forecast_data`)
            cross join
            (select distinct genericTimePeriod from `genericDataset.raw_forecast_data`)) a
            left join
            (select *, concat(genericHierarchy) as aggLevel from `genericDataset.raw_forecast_data`) b
            on a.aggLevel = b.aggLevel
            and a.genericTimePeriod = b.genericTimePeriod

            )

            
            select a.*, EXTRACT(DAYOFWEEK FROM genericTimePeriod) as dayofweek
            from TBL1 a
            ,
            (select min(genericTimePeriod) start_date, concat(genericHierarchy) as aggLevel
            from `genericDataset.raw_forecast_data`
            where genericForecastValue > 0
            group by 2) b
            where genericTimePeriod >= start_date
            and a.aggLevel = b.aggLevel
            );
            ''',replaceDict, 'createForecast')
    
    with out:
        print(datetime.datetime.now().strftime('%H:%M:%S'), 'Forecast 1 step 1/5 completed...')
    

    exec_sql(
    '''
    create or replace table `genericDataset.forecast_periods` genericTableOptionList AS (
    with periods as(
    SELECT A.FCST_genericTimePeriod as SNPSHT_genericTimePeriod, C.genericTimePeriod as FCST_genericTimePeriod, B.EVAL_genericTimePeriod
    , DATE_DIFF(A.fcst_genericTimePeriod
    , date_add(B.eval_genericTimePeriod
    , interval mod(greatest(date_diff(date_sub(a.fcst_genericTimePeriod, interval DATE_DIFF(A.fcst_genericTimePeriod, B.eval_genericTimePeriod,YEAR) YEAR), b.eval_genericTimePeriod,genericFrequency)
    ,date_diff(date_sub(a.fcst_genericTimePeriod, interval DATE_DIFF(A.fcst_genericTimePeriod, B.eval_genericTimePeriod,YEAR) - 1 YEAR), b.eval_genericTimePeriod,genericFrequency)),52) genericFrequency) ,YEAR) AS YR
    , mod(greatest(date_diff(date_sub(a.fcst_genericTimePeriod, interval DATE_DIFF(A.fcst_genericTimePeriod, B.eval_genericTimePeriod,YEAR) YEAR), b.eval_genericTimePeriod,genericFrequency)
    ,date_diff(date_sub(a.fcst_genericTimePeriod, interval DATE_DIFF(A.fcst_genericTimePeriod, B.eval_genericTimePeriod,YEAR) - 1 YEAR), b.eval_genericTimePeriod,genericFrequency)), 52)
    AS genericFrequency
    FROM
    (
    SELECT DISTINCT RANK() OVER (ORDER BY genericTimePeriod) TIME_RANK, genericTimePeriod FCST_genericTimePeriod, FSCL_YR FROM `pr-edw-views-thd.SHARED.CAL_PRD_HIER_FD`
    WHERE cal_dt BETWEEN (select min(genericTimePeriod) from `genericDataset.forecast_data`) and current_date()
    and cal_dt = genericCalendarColumn
    ORDER BY 1
    ) A
    INNER JOIN
    (
    SELECT DISTINCT RANK() OVER (ORDER BY genericTimePeriod) TIME_RANK, genericTimePeriod EVAL_genericTimePeriod FROM `pr-edw-views-thd.SHARED.CAL_PRD_HIER_FD`
    WHERE cal_dt BETWEEN (select min(genericTimePeriod) from `genericDataset.forecast_data`) and (select max(genericTimePeriod) from `genericDataset.forecast_data`)
    and cal_dt = genericCalendarColumn
    ORDER BY 1
    ) B
    ON (date_diff(date_sub(a.fcst_genericTimePeriod, interval DATE_DIFF(A.fcst_genericTimePeriod, B.eval_genericTimePeriod,YEAR) YEAR), b.eval_genericTimePeriod,genericFrequency) BETWEEN 1 AND genericNumPeriods/4
    or date_diff(date_sub(a.fcst_genericTimePeriod, interval DATE_DIFF(A.fcst_genericTimePeriod, B.eval_genericTimePeriod,YEAR) - 1 YEAR), b.eval_genericTimePeriod,genericFrequency) BETWEEN 1 AND genericNumPeriods/4)
    INNER JOIN
    (SELECT DISTINCT RANK() OVER (ORDER BY genericTimePeriod) TIME_RANK, genericTimePeriod FROM `pr-edw-views-thd.SHARED.CAL_PRD_HIER_FD`
    WHERE cal_dt >= (select min(genericTimePeriod) from `genericDataset.forecast_data`)
    and cal_dt = genericCalendarColumn
    ORDER BY 1) C
    ON date_diff(C.genericTimePeriod, A.fcst_genericTimePeriod, genericFrequency) BETWEEN 0 AND (snapshotPeriods - 1)
    WHERE
    a.fcst_genericTimePeriod in (SELECT distinct genericTimePeriod From `pr-edw-views-thd.SHARED.CAL_PRD_HIER_FD` where 
    CAST(cast('forecastPeriodStart' as string) as date) <= CAL_DT
    and CAST(cast('forecastPeriodEnd' as string) as date) >= CAL_DT)
    and A.TIME_RANK > B.TIME_RANK
    and DATE_DIFF(A.fcst_genericTimePeriod, B.eval_genericTimePeriod,YEAR) <= 6#years of history to use
    )
    
    select * from periods
    where concat(cast(snpsht_genericTimePeriod as string), cast(fcst_genericTimePeriod as string), cast(yr as string)) in
    (select concat(cast(snpsht_genericTimePeriod as string), cast(fcst_genericTimePeriod as string), cast(yr as string)) from periods
    group by 1
    having count(genericFrequency) = genericNumPeriods/4)
    );
    ''',replaceDict, 'createForecast'
    ,altSQL='''create or replace table `genericDataset.forecast_periods` genericTableOptionList AS (
    with periods as(
    SELECT A.FCST_genericTimePeriod as SNPSHT_genericTimePeriod, C.genericTimePeriod as FCST_genericTimePeriod, B.EVAL_genericTimePeriod
    , DATE_DIFF(A.fcst_genericTimePeriod
    , date_add(B.eval_genericTimePeriod
    , interval mod(greatest(date_diff(date_sub(a.fcst_genericTimePeriod, interval DATE_DIFF(A.fcst_genericTimePeriod, B.eval_genericTimePeriod,YEAR) YEAR), b.eval_genericTimePeriod,genericFrequency)
    ,date_diff(date_sub(a.fcst_genericTimePeriod, interval DATE_DIFF(A.fcst_genericTimePeriod, B.eval_genericTimePeriod,YEAR) - 1 YEAR), b.eval_genericTimePeriod,genericFrequency)),52) genericFrequency) ,YEAR) AS YR
    , mod(greatest(date_diff(date_sub(a.fcst_genericTimePeriod, interval DATE_DIFF(A.fcst_genericTimePeriod, B.eval_genericTimePeriod,YEAR) YEAR), b.eval_genericTimePeriod,genericFrequency)
    ,date_diff(date_sub(a.fcst_genericTimePeriod, interval DATE_DIFF(A.fcst_genericTimePeriod, B.eval_genericTimePeriod,YEAR) - 1 YEAR), b.eval_genericTimePeriod,genericFrequency)), 52)
    AS genericFrequency
    FROM
    (
    SELECT DISTINCT RANK() OVER (ORDER BY genericTimePeriod) TIME_RANK, genericTimePeriod FCST_genericTimePeriod, FSCL_YR FROM `pr-edw-views-thd.SHARED.CAL_PRD_HIER_FD`
    WHERE cal_dt BETWEEN (select min(genericTimePeriod) from `genericDataset.forecast_data`) and current_date()
    and cal_dt = genericCalendarColumn
    ORDER BY 1
    ) A
    INNER JOIN
    (
    SELECT DISTINCT RANK() OVER (ORDER BY genericTimePeriod) TIME_RANK, genericTimePeriod EVAL_genericTimePeriod FROM `pr-edw-views-thd.SHARED.CAL_PRD_HIER_FD`
    WHERE cal_dt BETWEEN (select min(genericTimePeriod) from `genericDataset.forecast_data`) and (select max(genericTimePeriod) from `genericDataset.forecast_data`)
    and cal_dt = genericCalendarColumn
    ORDER BY 1
    ) B
    ON (date_diff(date_sub(a.fcst_genericTimePeriod, interval DATE_DIFF(A.fcst_genericTimePeriod, B.eval_genericTimePeriod,YEAR) YEAR), b.eval_genericTimePeriod,genericFrequency) BETWEEN 1 AND genericNumPeriods/4
    or date_diff(date_sub(a.fcst_genericTimePeriod, interval DATE_DIFF(A.fcst_genericTimePeriod, B.eval_genericTimePeriod,YEAR) - 1 YEAR), b.eval_genericTimePeriod,genericFrequency) BETWEEN 1 AND genericNumPeriods/4)
    INNER JOIN
    (SELECT DISTINCT RANK() OVER (ORDER BY genericTimePeriod) TIME_RANK, genericTimePeriod FROM `pr-edw-views-thd.SHARED.CAL_PRD_HIER_FD`
    WHERE cal_dt >= (select min(genericTimePeriod) from `genericDataset.forecast_data`)
    and cal_dt = genericCalendarColumn
    ORDER BY 1) C
    ON date_diff(C.genericTimePeriod, A.fcst_genericTimePeriod, genericFrequency) BETWEEN 0 AND (snapshotPeriods - 1)
    WHERE
    a.fcst_genericTimePeriod in (SELECT distinct genericTimePeriod From `pr-edw-views-thd.SHARED.CAL_PRD_HIER_FD` where 
    date_trunc(current_date(), genericFrequency) <= CAL_DT
    and date_trunc(current_date(), genericFrequency) >= CAL_DT)
    and A.TIME_RANK > B.TIME_RANK
    and DATE_DIFF(A.fcst_genericTimePeriod, B.eval_genericTimePeriod,YEAR) <= 6#years of history to use
    )
    
    select * from periods
    where concat(cast(snpsht_genericTimePeriod as string), cast(fcst_genericTimePeriod as string), cast(yr as string)) in
    (select concat(cast(snpsht_genericTimePeriod as string), cast(fcst_genericTimePeriod as string), cast(yr as string)) from periods
    group by 1
    having count(genericFrequency) = genericNumPeriods/4)
    );''')

    with out:
        print(datetime.datetime.now().strftime('%H:%M:%S'), 'Forecast 1 step 2/5 completed...')

    exec_sql(
    '''
    create or replace table `genericDataset.forecast_samples` genericTableOptionList AS (
    SELECT
            snpsht_genericTimePeriod
            , fcst_genericTimePeriod
            , genericTimePeriod
            , aggLevel
            , genericForecastValue
            , SAFE_DIVIDE(genericForecastValue , SUM(genericForecastValue) 
    OVER (PARTITION BY SNPSHT_genericTimePeriod,fcst_genericTimePeriod, cast(YR as int64), aggLevel)) KEYREC_VAL
    from `genericDataset.forecast_data` a
    inner join
    `genericDataset.forecast_periods` b
    on a.genericTimePeriod = b.eval_genericTimePeriod
    );
    ''',replaceDict, 'createForecast')
    
    with out:
        print(datetime.datetime.now().strftime('%H:%M:%S'), 'Forecast 1 step 3/5 completed...')


    exec_sql(
     '''
    create or replace table `genericDataset.value_comparison_source` genericTableOptionList as (
    with
    TBL2B AS (
    SELECT t1.SNPSHT_genericTimePeriod, t1.FCST_genericTimePeriod,
    t1.EVAL_genericTimePeriod AS TY_EVAL_genericTimePeriod, t2.EVAL_genericTimePeriod AS PREV_EVAL_genericTimePeriod, T2.YR, t3.*
    FROM `genericDataset.forecast_periods` t1
    INNER JOIN `genericDataset.forecast_periods` t2
      ON T1.FCST_genericTimePeriod = T2.FCST_genericTimePeriod AND T1.genericFrequency = T2.genericFrequency
      and t1.SNPSHT_genericTimePeriod = t2.SNPSHT_genericTimePeriod
     cross join (select distinct aggLevel from `genericDataset.forecast_samples`) t3
    WHERE T1.YR = 0 AND T2.YR <> 0
    ),

    Data_Start as (
    select     aggLevel 
            , min(genericTimePeriod) as Start_Date
            From `genericDataset.forecast_data`
            group by 1)
    ,
    comp_calc as (
    select
    t1.SNPSHT_genericTimePeriod
    ,t1.fcst_genericTimePeriod
    ,t1.aggLevel, t1.yr
    , sum(current_vals.genericForecastValue) as current_genericForecastValue
    , sum(previous_vals.genericForecastValue) as prev_genericForecastValue 
    ,coalesce(case 
    when sum(current_vals.genericForecastValue) <= genericNumPeriods/4 and sum(previous_vals.genericForecastValue) <= genericNumPeriods/4--low volume option (reduce impact of comp)
    then SAFE_DIVIDE(avg(current_vals.genericForecastValue),avg(previous_vals.genericForecastValue))
    when sum(current_vals.genericForecastValue) < sum(previous_vals.genericForecastValue)--trending down (take normal values)
    then SAFE_DIVIDE(avg(current_vals.genericForecastValue),avg(previous_vals.genericForecastValue))
    else SAFE_DIVIDE(avg(current_vals.genericForecastValue),avg(greatest(previous_vals.genericForecastValue,1)))--trending up (minimum value for prev yrs)
    end,1) as COMP_FCTR--new comp cap if too many 0 values
    , SAFE_DIVIDE(1 , SUM(POWER(coalesce(current_vals.KEYREC_VAL, 0) - coalesce(previous_vals.KEYREC_VAL,0), 2))) YR_DIFF_SQRD
    from TBL2B t1
    left join `genericDataset.forecast_samples` current_vals
    on t1.TY_EVAL_genericTimePeriod = current_vals.genericTimePeriod 
    and t1.fcst_genericTimePeriod = current_vals.fcst_genericTimePeriod
    and current_vals.aggLevel = t1.aggLevel
    and t1.SNPSHT_genericTimePeriod = current_vals.snpsht_genericTimePeriod
    left join `genericDataset.forecast_samples` previous_vals
    on T1.PREV_EVAL_genericTimePeriod = previous_vals.genericTimePeriod
    and t1.fcst_genericTimePeriod = previous_vals.fcst_genericTimePeriod
    and t1.aggLevel = previous_vals.aggLevel
    and t1.SNPSHT_genericTimePeriod = previous_vals.snpsht_genericTimePeriod
    inner join Data_Start start
    on t1.aggLevel = start.aggLevel
    group by 1,2,3,4, start.Start_Date
    having min(t1.PREV_EVAL_genericTimePeriod) >= start.Start_Date
    )
    
    select 
    *
    from comp_calc
    
    );
    ''',replaceDict, 'createForecast')
    
    with out:
        print(datetime.datetime.now().strftime('%H:%M:%S'), 'Forecast 1 step 4/5 completed...')
    
    #split aggLevel column back out into original columns
    columns = []
    for i in range(len(hierarchyColumns)):
        columns.append('''split(a.aggLevel, '|')[OFFSET({})] as {}'''.format(i, hierarchyColumns[i]))
    deagged = ', '.join(columns)

    exec_sql(
    '''
    create or replace table `genericDataset.genericOutputTable` as (

    with
    /*avg_comp as (
    select
    t1.SNPSHT_genericTimePeriod as snpsht_wk
    ,t2.fcst_genericTimePeriod as fcst_wk, t1.aggLevel, t1.yr, avg(t1.comp_fctr) as avg_comp
    from `genericDataset.value_comparison_source` t1--replacement vals
    inner join `genericDataset.value_comparison_source` t2--val to be replaced
    on t1.fcst_genericTimePeriod < t2.fcst_genericTimePeriod
    and t1.aggLevel = t2.aggLevel
    and t1.yr = t2.yr
    --and t2.comp_fctr > CASE WHEN power(2,t2.yr) <= 5.1 THEN power(2,t2.yr) ELSE 5.1 END
    --and t1.comp_fctr <= CASE WHEN power(2,t2.yr) <= 5.1 THEN power(2,t2.yr) ELSE 5.1 END
    and t2.comp_fctr > t2.percentile_95
    and t1.comp_fctr <= t2.percentile_95
    group by 1,2,3,4
    ),*/
    avg_comp as (
    select distinct a.*
    ,PERCENTILE_CONT(b.comp_fctr, .1) OVER(PARTITION BY a.snpsht_genericTimePeriod, a.AGGLEVEL) as percentile_10
    ,PERCENTILE_CONT(b.comp_fctr, .5) OVER(PARTITION BY a.snpsht_genericTimePeriod, a.AGGLEVEL) as median
    ,PERCENTILE_CONT(b.comp_fctr, .9) OVER(PARTITION BY a.snpsht_genericTimePeriod, a.AGGLEVEL) as percentile_90
    from
    `genericDataset.value_comparison_source` a
    inner join
    `genericDataset.value_comparison_source` b
    on a.snpsht_genericTimePeriod > b.snpsht_genericTimePeriod
    and b.snpsht_genericTimePeriod > date_sub(a.snpsht_genericTimePeriod, interval 1 year)
    and date_diff(a.snpsht_genericTimePeriod, a.fcst_genericTimePeriod, day) = date_diff(b.snpsht_genericTimePeriod, b.fcst_genericTimePeriod, day)
    and a.aggLevel = b.aggLevel
    ),

    genericForecastValue_comparison as (
    select 
    a.SNPSHT_genericTimePeriod
    ,a.fcst_genericTimePeriod
    ,a.aggLevel, a.yr
    , a.current_genericForecastValue
    , a.prev_genericForecastValue 
    , coalesce(case when b.comp_fctr > b.percentile_90 then median
    --when b.comp_fctr < b.percentile_10 then median
    else b.comp_fctr end
    , a.comp_fctr) as comp_fctr
    , a.YR_DIFF_SQRD
    from `genericDataset.value_comparison_source` a
    left join
    avg_comp b
    on
    a.aggLevel = b.aggLevel
    and a.yr = b.yr
    and a.fcst_genericTimePeriod = b.fcst_genericTimePeriod
    and a.SNPSHT_genericTimePeriod = b.snpsht_genericTimePeriod
    ),

    weights as (
    SELECT
    A.SNPSHT_genericTimePeriod
    ,A.fcst_genericTimePeriod
    ,YR
    ,a.aggLevel
    ,A.yr_diff_sqrd
    ,b.ttl_yr_diff_sqrd
    , SAFE_DIVIDE(A.YR_DIFF_SQRD , B.TTL_YR_DIFF_SQRD) WEIGHT
    , A.comp_fctr
    from   genericForecastValue_comparison A
    left join
    (select SNPSHT_genericTimePeriod, fcst_genericTimePeriod, aggLevel
    ,sum(YR_DIFF_SQRD) ttl_YR_DIFF_SQRD
    from `genericDataset.value_comparison_source`
    group by 1,2,3) B
    on A.fcst_genericTimePeriod = B.fcst_genericTimePeriod
    and a.aggLevel = b.aggLevel
    and a.SNPSHT_genericTimePeriod = b.SNPSHT_genericTimePeriod
    ),

    years_history as (
    select SNPSHT_genericTimePeriod, fcst_genericTimePeriod, aggLevel, count(distinct yr) as years_available
    from weights
    group by 1,2,3
    ),

     missing_samples as(
        select * from `genericDataset.forecast_samples`
    where concat(cast(snpsht_genericTimePeriod as string), aggLevel) not in (select distinct concat(cast(snpsht_genericTimePeriod as string), aggLevel) from years_history)
    --find any aggs/week combos that are present in historic data, but not in forecast data (because dont have minimum 1 year + 13 period history)
    ),



    forecast as (
    select
    b.SNPSHT_genericTimePeriod
    ,b.fcst_genericTimePeriod
    ,b.aggLevel
    ,coalesce( forecastTransform1 sum(a.genericForecastValue * b.weight * b.comp_fctr) forecastTransform2 ,0) as forecast1_genericForecastValue--multi year history
    --,coalesce(sum(a.genericForecastValue * b.weight * b.comp_fctr),0) as exact_forecast1_genericForecastValue
    from `genericDataset.forecast_data` a
    inner join weights b



    --on date_diff(date_trunc((date_sub(b.fcst_genericTimePeriod, interval DATE_DIFF(b.fcst_genericTimePeriod, a.genericTimePeriod,YEAR) YEAR))
    --, week(a.weekday)), a.genericTimePeriod,genericFrequency) = 0

    on date_diff(date_sub(b.fcst_genericTimePeriod, interval DATE_DIFF(b.fcst_genericTimePeriod, a.genericTimePeriod,YEAR) YEAR), a.genericTimePeriod,week) = 0
    and extract(DAYOFWEEK from b.fcst_genericTimePeriod) = a.dayofweek



    --on MOD(DATE_DIFF(b.fcst_genericTimePeriod,a.genericTimePeriod,WEEK),52) = 0--multi year join
    and DATE_DIFF(b.fcst_genericTimePeriod, a.genericTimePeriod,YEAR) = b.yr
        AND a.aggLevel = b.aggLevel
    left join years_history history
    on b.snpsht_genericTimePeriod = history.snpsht_genericTimePeriod
        and b.fcst_genericTimePeriod = history.fcst_genericTimePeriod
        and history.aggLevel = b.aggLevel
    where history.years_available > 1    
    group by 1,2,3
    union all
    select
    b.SNPSHT_genericTimePeriod
    ,b.fcst_genericTimePeriod
    ,b.aggLevel
    ,coalesce( forecastTransform1 b.avg_genericForecastValue forecastTransform2 ,0) as DS_genericForecastValue
    --,coalesce(b.avg_genericForecastValue,0) as exact_DS_genericForecastValue
    from
    (SELECT t3.snpsht_genericTimePeriod, t3.fcst_genericTimePeriod
    ,t3.aggLevel
    ,avg(t3.genericForecastValue) as avg_genericForecastValue
    FROM  `genericDataset.forecast_periods` t1
    inner join `genericDataset.forecast_samples` t3
    on t1.snpsht_genericTimePeriod = t3.snpsht_genericTimePeriod
    and t1.fcst_genericTimePeriod = t3.fcst_genericTimePeriod
    and t1.eval_genericTimePeriod = t3.genericTimePeriod
    WHERE T1.YR = 0
    group by 1,2,3) b
    left join years_history history
    on b.snpsht_genericTimePeriod = history.snpsht_genericTimePeriod
        and b.fcst_genericTimePeriod = history.fcst_genericTimePeriod
        and history.aggLevel = b.aggLevel
    where history.years_available <= 1
    union all
    select snpsht_genericTimePeriod, fcst_genericTimePeriod, aggLevel
    , coalesce( forecastTransform1 avg(genericForecastValue) forecastTransform2 ,0) as forecast1_genericForecastValue
    --, coalesce(avg(genericForecastValue),0) as exact_forecast1_genericForecastValue
    from missing_samples
    group by 1,2,3
    )

    select
    a.*, cast(0.0 as float64) as forecast2_genericForecastValue--, cast(null as float64) as exact_forecast2_genericForecastValue
    , cast(0.0 as float64) as forecast3_genericForecastValue--, cast(null as float64) as exact_forecast3_genericForecastValue
    , cast(null as float64) as forecast4_genericForecastValue
    , cast(null as float64) as one_forecast_genericForecastValue
    , cast(b.genericForecastValue as float64) as actual_genericForecastValue
    ,{}
    from forecast a
    left join `genericDataset.forecast_data` b
    on a.fcst_genericTimePeriod = b.genericTimePeriod
    and a.aggLevel = b.aggLevel

    );
    '''.format(deagged),replaceDict, 'createForecast'
    ,altSQL='''
    create or replace table `genericDataset.KERNEL_FCST` genericTableOptionList as (
    with
    avg_comp as (
    select
    t1.SNPSHT_genericTimePeriod as snpsht_wk
    ,t2.fcst_genericTimePeriod as fcst_wk, t1.aggLevel, t1.yr, avg(t1.comp_fctr) as avg_comp
    from `genericDataset.value_comparison_source` t1--replacement vals
    inner join `genericDataset.value_comparison_source` t2--val to be replaced
    on t1.fcst_genericTimePeriod < t2.fcst_genericTimePeriod
    and t1.aggLevel = t2.aggLevel
    and t1.yr = t2.yr
    and t2.comp_fctr > CASE WHEN power(2,t2.yr) <= 5.1 THEN power(2,t2.yr) ELSE 5.1 END
    and t1.comp_fctr <= CASE WHEN power(2,t2.yr) <= 5.1 THEN power(2,t2.yr) ELSE 5.1 END
    group by 1,2,3,4
    ),

    genericForecastValue_comparison as (
    select 
    a.SNPSHT_genericTimePeriod
    ,a.fcst_genericTimePeriod
    ,a.aggLevel, a.yr
    , a.current_genericForecastValue
    , a.prev_genericForecastValue 
    , a.comp_fctr
    , a.YR_DIFF_SQRD
    from `genericDataset.value_comparison_source` a
    left join
    avg_comp b
    on
    a.aggLevel = b.aggLevel
    and a.yr = b.yr
    and a.fcst_genericTimePeriod = b.fcst_wk
    and a.SNPSHT_genericTimePeriod = b.snpsht_wk
    ),

    weights as (
    SELECT
    A.SNPSHT_genericTimePeriod
    ,A.fcst_genericTimePeriod
    ,YR
    ,a.aggLevel
    ,A.yr_diff_sqrd
    ,b.ttl_yr_diff_sqrd
    , SAFE_DIVIDE(A.YR_DIFF_SQRD , B.TTL_YR_DIFF_SQRD) WEIGHT
    , A.comp_fctr
    from   genericForecastValue_comparison A
    left join
    (select SNPSHT_genericTimePeriod, fcst_genericTimePeriod, aggLevel
    ,sum(YR_DIFF_SQRD) ttl_YR_DIFF_SQRD
    from `genericDataset.value_comparison_source`
    group by 1,2,3) B
    on A.fcst_genericTimePeriod = B.fcst_genericTimePeriod
    and a.aggLevel = b.aggLevel
    and a.SNPSHT_genericTimePeriod = b.SNPSHT_genericTimePeriod
    ),

    years_history as (
    select SNPSHT_genericTimePeriod, fcst_genericTimePeriod, aggLevel, count(distinct yr) as years_available
    from weights
    group by 1,2,3
    ),

     missing_samples as(
        select * from `genericDataset.forecast_samples`
    where concat(cast(snpsht_genericTimePeriod as string), aggLevel) not in (select distinct concat(cast(snpsht_genericTimePeriod as string), aggLevel) from years_history)
    --find any aggs/week combos that are present in historic data, but not in forecast data (because dont have minimum 1 year + 13 period history)
    ),



    forecast as (
    select
    b.SNPSHT_genericTimePeriod
    ,b.fcst_genericTimePeriod
    ,b.aggLevel
    ,coalesce( forecastTransform1 sum(a.genericForecastValue * b.weight * b.comp_fctr) forecastTransform2 ,0) as forecast1_genericForecastValue--multi year history
    --,coalesce(sum(a.genericForecastValue * b.weight * b.comp_fctr),0) as exact_forecast1_genericForecastValue
    from `genericDataset.forecast_data` a
    inner join weights b



    --on date_diff(date_trunc((date_sub(b.fcst_genericTimePeriod, interval DATE_DIFF(b.fcst_genericTimePeriod, a.genericTimePeriod,YEAR) YEAR))
    --, week(a.weekday)), a.genericTimePeriod,genericFrequency) = 0

    on date_diff(date_sub(b.fcst_genericTimePeriod, interval DATE_DIFF(b.fcst_genericTimePeriod, a.genericTimePeriod,YEAR) YEAR), a.genericTimePeriod,week) = 0
    and extract(DAYOFWEEK from b.fcst_genericTimePeriod) = a.dayofweek



    --on MOD(DATE_DIFF(b.fcst_genericTimePeriod,a.genericTimePeriod,WEEK),52) = 0--multi year join
    and DATE_DIFF(b.fcst_genericTimePeriod, a.genericTimePeriod,YEAR) = b.yr
        AND a.aggLevel = b.aggLevel
    left join years_history history
    on b.snpsht_genericTimePeriod = history.snpsht_genericTimePeriod
        and b.fcst_genericTimePeriod = history.fcst_genericTimePeriod
        and history.aggLevel = b.aggLevel
    where history.years_available > 1    
    group by 1,2,3
    union all
    select
    b.SNPSHT_genericTimePeriod
    ,b.fcst_genericTimePeriod
    ,b.aggLevel
    ,coalesce( forecastTransform1 b.avg_genericForecastValue forecastTransform2 ,0) as DS_genericForecastValue
    --,coalesce(b.avg_genericForecastValue,0) as exact_DS_genericForecastValue
    from
    (SELECT t3.snpsht_genericTimePeriod, t3.fcst_genericTimePeriod
    ,t3.aggLevel
    ,avg(t3.genericForecastValue) as avg_genericForecastValue
    FROM  `genericDataset.forecast_periods` t1
    inner join `genericDataset.forecast_samples` t3
    on t1.snpsht_genericTimePeriod = t3.snpsht_genericTimePeriod
    and t1.fcst_genericTimePeriod = t3.fcst_genericTimePeriod
    and t1.eval_genericTimePeriod = t3.genericTimePeriod
    WHERE T1.YR = 0
    group by 1,2,3) b
    left join years_history history
    on b.snpsht_genericTimePeriod = history.snpsht_genericTimePeriod
        and b.fcst_genericTimePeriod = history.fcst_genericTimePeriod
        and history.aggLevel = b.aggLevel
    where history.years_available <= 1
    union all
    select snpsht_genericTimePeriod, fcst_genericTimePeriod, aggLevel
    , coalesce( forecastTransform1 avg(genericForecastValue) forecastTransform2 ,0) as forecast1_genericForecastValue
    --, coalesce(avg(genericForecastValue),0) as exact_forecast1_genericForecastValue
    from missing_samples
    group by 1,2,3
    )

    select
    a.*, cast(null as float64) as forecast2_genericForecastValue--, cast(null as float64) as exact_forecast2_genericForecastValue
    , cast(null as float64) as forecast3_genericForecastValue--, cast(null as float64) as exact_forecast3_genericForecastValue
    , cast(null as float64) as forecast4_genericForecastValue
    , cast(null as float64) as one_forecast_genericForecastValue
    , cast(b.genericForecastValue as float64) as actual_genericForecastValue
    ,{}
    from forecast a
    left join `genericDataset.forecast_data` b
    on a.fcst_genericTimePeriod = b.genericTimePeriod
    and a.aggLevel = b.aggLevel);
    
    
    
    create or replace table `genericDataset.genericOutputTable_copy` as (
    select * from `genericDataset.genericOutputTable`
    );
    
    create or replace table `genericDataset.genericOutputTable` as (
    select * from `genericDataset.genericOutputTable_copy`
    union distinct
    select * from `genericDataset.KERNEL_FCST` where snpsht_genericTimePeriod = date_trunc(current_date(), genericFrequency)
    
    );

    '''.format(deagged)
    )

    with out:
        print(datetime.datetime.now().strftime('%H:%M:%S'), 'Forecast 1 step 5/5 completed, forecast created successfully...')
    display(Javascript("execCellsByTag('Forecast2')"))
except:
    with out:
        print(datetime.datetime.now().strftime('%H:%M:%S'), 'Error: Invalid input detected while generating forecast 1. Please verify inputs and try again.')


In [ ]:

try:
    exec_sql('''
    CREATE OR REPLACE TABLE `genericDataset.BQML_SEASONAL_CALENDAR` genericTableOptionList AS (
    WITH TBL1 AS (
    SELECT *
      , MAX(DAILY_HOLIDAY) OVER (PARTITION BY FSCL_WK_END_DT) AS WEEKLY_HOLIDAY
      , CONCAT('BEFORE ', LEAD(DAILY_HOLIDAY) OVER (ORDER BY CAL_DT)) AS LEAD_DAILY_HOLIDAY
      , CONCAT('AFTER ', LAG(DAILY_HOLIDAY) OVER (ORDER BY CAL_DT)) AS LAG_DAILY_HOLIDAY
    FROM (
      SELECT *
        , CASE WHEN (MONTH = 1 AND DAY = 1 AND DAYOFWEEK BETWEEN 2 AND 6) OR (MONTH = 1 AND DAY = 2 AND DAYOFWEEK = 2) OR (MONTH = 12 AND DAY = 31 AND DAYOFWEEK = 6) THEN 'NEW YEARS'
            --WHEN MONTH = 5 AND DAY BETWEEN 25 AND 31 AND DAYOFWEEK = 2 THEN 0
            --WHEN (MONTH = 7 AND DAY = 4 AND DAYOFWEEK BETWEEN 2 AND 6) OR (MONTH = 7 AND DAY = 5 AND DAYOFWEEK = 2) OR (MONTH = 7 AND DAY = 3 AND DAYOFWEEK = 6) THEN 0
            --WHEN MONTH = 9 AND DAY BETWEEN 1 AND 7 AND DAYOFWEEK = 2 THEN 0
            WHEN MONTH = 11 AND DAY BETWEEN 22 AND 28 AND DAYOFWEEK = 5 THEN 'THANKSGIVING'
            WHEN (MONTH = 12 AND DAY = 25 AND DAYOFWEEK BETWEEN 2 AND 6) OR (MONTH = 12 AND DAY = 26 AND DAYOFWEEK = 2) OR (MONTH = 12 AND DAY = 24 AND DAYOFWEEK = 6) THEN 'CHRISTMAS'
            ELSE NULL END AS DAILY_HOLIDAY
      FROM (
        SELECT *
          , EXTRACT(MONTH FROM CAL_DT) AS MONTH, EXTRACT(DAY FROM CAL_DT) AS DAY, EXTRACT(DAYOFWEEK FROM CAL_DT) AS DAYOFWEEK
        FROM `pr-edw-views-thd.SHARED.CAL_PRD_HIER_FD`
      )
    )
    )
    , TBL2 AS (
    SELECT *
      , CONCAT('BEFORE ', LEAD(WEEKLY_HOLIDAY) OVER (ORDER BY FSCL_WK_END_DT)) AS LEAD_WEEKLY_HOLIDAY
      , CONCAT('AFTER ', LAG(WEEKLY_HOLIDAY) OVER (ORDER BY FSCL_WK_END_DT)) AS LAG_WEEKLY_HOLIDAY
    FROM (SELECT DISTINCT FSCL_WK_END_DT, WEEKLY_HOLIDAY FROM TBL1)
    )
    , TBL3 AS (
    SELECT A.CAL_DT, A.FSCL_WK_END_DT, A.FSCL_PRD_END_DT
      , COALESCE(A.DAILY_HOLIDAY, A.LEAD_DAILY_HOLIDAY, A.LAG_DAILY_HOLIDAY, CAST(A.DAYOFWEEK AS STRING)) AS DAILY_SEASON
      , CASE WHEN A.DAILY_HOLIDAY IS NULL AND A.LEAD_DAILY_HOLIDAY IS NULL AND A.LAG_DAILY_HOLIDAY IS NULL THEN FALSE ELSE TRUE END AS DAILY_HOLIDAY_FLG
      , COALESCE(A.WEEKLY_HOLIDAY, B.LEAD_WEEKLY_HOLIDAY, B.LAG_WEEKLY_HOLIDAY, CAST(CASE WHEN A.FSCL_WK_NBR = 53 THEN 52 ELSE FSCL_WK_NBR END AS STRING)) AS WEEKLY_SEASON
      , CASE WHEN A.WEEKLY_HOLIDAY IS NULL AND B.LEAD_WEEKLY_HOLIDAY IS NULL AND B.LAG_WEEKLY_HOLIDAY IS NULL THEN FALSE ELSE TRUE END AS WEEKLY_HOLIDAY_FLG
      , FSCL_PRD_NBR AS MONTHLY_SEASON
    FROM TBL1 A
    INNER JOIN TBL2 B
      ON A.FSCL_WK_END_DT = B.FSCL_WK_END_DT
    )
    , TBL4 AS (
    SELECT *
        , SEASONALITY[OFFSET(0)].HOLIDAY_FLG AS HOLIDAY_FLG
    FROM (
    SELECT CAL_DT AS DT
        , genericSeasonality AS SEASONALITY
    FROM TBL3
    WHERE CAL_DT = genericCalendarColumn
    )
    )
    SELECT *
        , ARRAY_AGG(DT) OVER (ORDER BY DT ROWS BETWEEN 1 FOLLOWING AND snapshotPeriods FOLLOWING) AS FCST_DTS
        , ROW_NUMBER() OVER (ORDER BY DT) AS DT_RNK
        , LEAD(DT) OVER (ORDER BY DT) AS NEXT_DT
    FROM TBL4
    );
    ''',replaceDict, 'createForecast')
    with out:
         print(datetime.datetime.now().strftime('%H:%M:%S'), 'Forecast 2 step 1/9 completed...')
    
    exec_sql('''
    CREATE OR REPLACE TABLE `genericDataset.BQML_SEASONAL_SMOOTHING_CALENDAR` genericTableOptionList AS (
    WITH TYPES AS (
    genericSeasonalUnion
    )
    , NON_HOLIDAY_CAL AS (
    SELECT *
        , ROW_NUMBER() OVER (ORDER BY DT) AS NON_HOLIDAY_DT_RNK
    FROM `genericDataset.BQML_SEASONAL_CALENDAR`
    WHERE NOT HOLIDAY_FLG
    )
    SELECT B.TYPE, A.DT, C.DT AS SMOOTH_DT
    FROM NON_HOLIDAY_CAL A
    CROSS JOIN TYPES B
    CROSS JOIN NON_HOLIDAY_CAL C
    WHERE C.NON_HOLIDAY_DT_RNK BETWEEN A.NON_HOLIDAY_DT_RNK - (B.LAGS - 1) / 2 AND A.NON_HOLIDAY_DT_RNK + (B.LAGS - 1) / 2
    
    UNION ALL
    
    SELECT B.TYPE, A.DT, A.DT AS SMOOTH_DT
    FROM `genericDataset.BQML_SEASONAL_CALENDAR` A
    CROSS JOIN TYPES B
    WHERE HOLIDAY_FLG
    );
    ''',replaceDict, 'createForecast')
    with out:
        print(datetime.datetime.now().strftime('%H:%M:%S'), 'Forecast 2 step 2/9 completed...')
    
    exec_sql('''
    /*CREATE TEMP TABLE TBL1 AS (
    SELECT CONCAT(genericHierarchy) AS KEY
        , genericTimePeriod AS DT
        , SUM(genericForecastValue) AS QTY
    FROM `genericInputTable`
    GROUP BY 1,2
    );*/

    CREATE OR REPLACE TABLE `genericDataset.BQML_DATA` genericTableOptionList AS (
    WITH TBL1 AS (
        SELECT CONCAT(genericHierarchy) AS KEY
        , genericTimePeriod AS DT
        , SUM(genericForecastValue) AS QTY
        FROM `genericInputTable`
        GROUP BY 1,2
    ),
    KEYS AS ( --CONTROL FOR KEYS APPEARING MIDWAY THROUGH DATA BY TAKING KEY SPECIFIC MIN
    SELECT A.KEY, MIN(B.DT_RNK) AS MIN_DT_RNK
    FROM TBL1 A
    INNER JOIN `genericDataset.BQML_SEASONAL_CALENDAR` B
        ON A.DT = B.DT
    WHERE A.QTY > 0
    GROUP BY 1
    )
    , MAX_DT AS ( --WANT ALL KEYS TO END AT SAME POINT, SO DON'T TAKE KEY SPECIFIC MAX
    SELECT MAX(B.DT_RNK) AS MAX_DT_RNK
    FROM TBL1 A
    INNER JOIN `genericDataset.BQML_SEASONAL_CALENDAR` B
        ON A.DT = B.DT
    )
    , TBL2 AS (
    SELECT A.KEY
        , C.DT
        , C.SEASONALITY
        , C.FCST_DTS
        , C.NEXT_DT
        , C.DT_RNK BETWEEN A.MIN_DT_RNK AND B.MAX_DT_RNK AS FCST_CRT_DT_FLG --PUSH DATA OUT TO COMPUTE SEASONALITY VALUES FOR ALL NEEDED WEEKS, THIS IS HOW TO RECOVER WEEKS WHERE WE CREATE A FORECAST
        , CASE WHEN C.DT_RNK BETWEEN A.MIN_DT_RNK AND B.MAX_DT_RNK THEN COALESCE(D.QTY, 0.0) ELSE NULL END AS QTY
    FROM KEYS A
    CROSS JOIN MAX_DT B
    CROSS JOIN `genericDataset.BQML_SEASONAL_CALENDAR` C
    LEFT JOIN TBL1 D
        ON A.KEY = D.KEY AND C.DT = D.DT
    WHERE C.DT_RNK BETWEEN A.MIN_DT_RNK - (genericSeasonalLags - 1) / 2 AND B.MAX_DT_RNK + (genericSeasonalLags - 1) / 2 + snapshotPeriods
    )
    , TBL3 AS (
    SELECT *
        , LEAST(GREATEST(QTY, PERCENTILE_1), PERCENTILE_99) AS ADJ_QTY
    FROM (
        SELECT A.*
            , PERCENTILE_CONT(QTY, .99) OVER (PARTITION BY KEY) AS PERCENTILE_99
            , PERCENTILE_CONT(QTY, .01) OVER (PARTITION BY KEY) AS PERCENTILE_1
        FROM TBL2 A
    )
    )
    SELECT *
        , AVG(ADJ_QTY) OVER (PARTITION BY KEY ORDER BY DT ROWS BETWEEN genericNumPeriods PRECEDING AND 1 PRECEDING) AS PERIOD_AVG_QTY
        , COUNT(ADJ_QTY) OVER (PARTITION BY KEY ORDER BY DT ROWS BETWEEN genericNumPeriods PRECEDING AND 1 PRECEDING) AS PERIOD_COUNT_QTY
    FROM TBL3
    );
    ''',replaceDict, 'createForecast')
    with out:
        print(datetime.datetime.now().strftime('%H:%M:%S'), 'Forecast 2 step 3/9 completed...')

    exec_sql('''
    CREATE OR REPLACE TABLE `genericDataset.BQML_SEASONALITY` genericTableOptionList AS (
    SELECT * EXCEPT(SEASONAL_QTY)
        , SAFE_DIVIDE(SUM(SEASONAL_QTY) OVER (PARTITION BY KEY, TYPE, SEASON) - COALESCE(SEASONAL_QTY, 0), 
                      COUNT(SEASONAL_QTY) OVER (PARTITION BY KEY, TYPE, SEASON) - CASE WHEN SEASONAL_QTY IS NOT NULL THEN 1 ELSE 0 END) AS SEASONAL_QTY
            --SEASONAL ESTIMATE IS AVERAGE OF VALUES EXCLUDING CURRENT ROW - THAT WAY WE'RE KEEPING THE FORECAST FOR A DATE INDEPENDENT OF THE VALUE FOR THAT DATE
    FROM (
        SELECT A.KEY, A.DT
            , SEASONALITY.TYPE
            , SEASONALITY.SEASON
            , CASE WHEN PERIOD_COUNT_QTY = genericNumPeriods THEN A.ADJ_QTY - A.PERIOD_AVG_QTY ELSE NULL END AS SEASONAL_QTY
                --IF WE DON'T HAVE A FULL PERIOD OF DATA THEN OUR SEASONAL ESTIMATE WILL BE BIASED
        FROM `genericDataset.BQML_DATA` A
        CROSS JOIN UNNEST(A.SEASONALITY) AS SEASONALITY
    )
    );
    ''',replaceDict, 'createForecast')
    with out:
        print(datetime.datetime.now().strftime('%H:%M:%S'), 'Forecast 2 step 4/9 completed...')
    
    exec_sql('''
    CREATE TEMP FUNCTION DOT(V1 ANY TYPE, V2 ANY TYPE) AS ((
    --SELECT SUM(X1 * V2[OFFSET(I)])
    SELECT SUM(X1 * CASE WHEN IS_NAN(V2[OFFSET(I)]) THEN NULL ELSE V2[OFFSET(I)] END)
    FROM UNNEST(V1) X1 WITH OFFSET I
    ));

    CREATE TEMP FUNCTION SMOOTH(A ANY TYPE, DECAY FLOAT64) AS ((
    SELECT DOT(ARRAY_AGG(WT / SUM_WTS ORDER BY I), A)
    FROM (
      SELECT *
        , SUM(WT) OVER () AS SUM_WTS
      FROM (
        SELECT I
          , POW(DECAY, ABS((ARRAY_LENGTH(A) - 1) / 2 - I)) AS WT
        FROM UNNEST(GENERATE_ARRAY(0, ARRAY_LENGTH(A) - 1)) I
      )
    )
    ));
    
    CREATE OR REPLACE TABLE `genericDataset.BQML_SEASONALITY_SMOOTHED` genericTableOptionList AS (
    SELECT KEY, DT
        , SUM(SMOOTH(SEASONAL_QTYS, .5)) AS SMOOTHED_SEASONAL_QTY
        , SUM(SMOOTH(SEASONAL_QTYS, 0)) AS SEASONAL_QTY --USING 0 HERE PROVIDES NO SMOOTHING AT ALL, USED FOR AGGRESSIVE FORECAST, ALSO A BACKUP IN CASE WE HAVE LITTLE HISTORY
    FROM (
        SELECT A.KEY, A.DT, A.TYPE
            , ARRAY_AGG(COALESCE(C.SEASONAL_QTY, CAST('NAN' AS FLOAT64)) ORDER BY B.DT) AS SEASONAL_QTYS
        FROM `genericDataset.BQML_SEASONALITY` A
        INNER JOIN `genericDataset.BQML_SEASONAL_SMOOTHING_CALENDAR` B
            ON A.TYPE = B.TYPE AND A.DT = B.DT
        LEFT JOIN `genericDataset.BQML_SEASONALITY` C
            ON A.KEY = C.KEY AND A.TYPE = C.TYPE AND B.SMOOTH_DT = C.DT
        GROUP BY 1,2,3
    )
    GROUP BY 1,2
    );
    ''',replaceDict, 'createForecast')
    with out:
        print(datetime.datetime.now().strftime('%H:%M:%S'), 'Forecast 2 step 5/9 completed...')
    
    exec_sql('''

    CREATE OR REPLACE TABLE `genericDataset.BQML_EXP_SMOOTH_DATA` genericTableOptionList AS (
    SELECT A.KEY
        , A.DT
        , A.NEXT_DT
        , A.FCST_DTS
        , A.FCST_CRT_DT_FLG
        , A.QTY
        , A.ADJ_QTY
        , COALESCE(B.SMOOTHED_SEASONAL_QTY, B.SEASONAL_QTY, 0) AS SMOOTHED_SEASONAL_QTY
        , A.ADJ_QTY - COALESCE(B.SMOOTHED_SEASONAL_QTY, B.SEASONAL_QTY, 0) AS DESEASONALIZED_ADJ_QTY
        , COALESCE(B.SEASONAL_QTY, 0) AS SEASONAL_QTY
        , A.ADJ_QTY - COALESCE(B.SEASONAL_QTY, 0) AS ST_DESEASONALIZED_ADJ_QTY
    FROM `genericDataset.BQML_DATA` A
    INNER JOIN `genericDataset.BQML_SEASONALITY_SMOOTHED` B
        ON A.KEY = B.KEY AND A.DT = B.DT
    );
    ''',replaceDict, 'createForecast')
    with out:
        print(datetime.datetime.now().strftime('%H:%M:%S'), 'Forecast 2 step 6/9 completed...')

    exec_sql('''
    CREATE TEMP FUNCTION EXP_SMOOTH(data ARRAY<STRUCT<dt DATE, deseasonalized_adj_qty FLOAT64, st_deseasonalized_adj_qty FLOAT64>>)
    RETURNS STRUCT<data ARRAY<STRUCT<dt DATE, deseasonalized_adj_qty FLOAT64, st_deseasonalized_adj_qty FLOAT64, fcst_deseasonalized_qty FLOAT64, st_fcst_deseasonalized_qty FLOAT64>>, 
                   alpha FLOAT64, alpha_h1 FLOAT64, alpha_h2 FLOAT64, alpha_q1 FLOAT64, alpha_q2 FLOAT64, alpha_q3 FLOAT64, alpha_q4 FLOAT64>
    LANGUAGE js
    AS """
    function golden_section_search(f, lb, ub, tol) { //https://en.wikipedia.org/wiki/Golden-section_search
        var gr = 1.618033988
        var x1 = {x: lb, y: f(lb)}
        var x2 = {x: (ub + gr * lb) / (gr + 1)}
        x2.y = f(x2.x)
        var x3 = {x: ub, y: f(lb)}
        function search(f, x1, x2, x3, tol) {
            if (x3.x - x1.x < tol) {
                return (x1.x + x3.x) / 2
            }
            var x4 = {x: x1.x + (x3.x - x2.x)}
            x4.y = f(x4.x)
            if (x4.x > x2.x) {
                if (x4.y < x2.y) {
                    return search(f, x2, x4, x3, tol)
                } else {
                    return search(f, x1, x2, x4, tol)
                }
            }
            else {
                if (x4.y < x2.y) {
                    return search(f, x1, x4, x2, tol)
                } else {
                    return search(f, x4, x2, x3, tol)
                }
            }
        }
        return search(f, x1, x2, x3, tol)
    }
    
    function fit(alpha, input_col, sse_low, sse_high, output_col, output_low, output_high) {
        data[0].temp = data[0][input_col]
        var sse = 0
        data.forEach((d, i) => {
            if (i >= 1) {
                d.temp = alpha * data[i][input_col] + (1 - alpha) * data[i-1].temp
                if (i >= sse_low && i <= sse_high) {
                    sse += Math.pow(d[input_col] - data[i-1].temp, 2)
                }
                if (output_col != undefined && i >= output_low && i <= output_high) {
                    d[output_col] = d.temp
                }
            }
        })
        return {data: data, sse: sse}
    }
    
    h1 = Math.floor(data.length / 2)
    q1 = Math.floor(data.length / 4)
    q2 = q1 * 2
    q3 = q1 * 3
    last = data.length - 1
    
    alpha = golden_section_search(x => fit(x, 'deseasonalized_adj_qty', 0, last).sse, 0, 1, .01) //alpha optimized on all data
    alpha_h1 = golden_section_search(x => fit(x, 'deseasonalized_adj_qty', 0, h1).sse, 0, 1, .01) //alpha optimized for h1
    alpha_h2 = golden_section_search(x => fit(x, 'deseasonalized_adj_qty', h1 + 1, last).sse, 0, 1, .01) //etc.
    alpha_q1 = golden_section_search(x => fit(x, 'st_deseasonalized_adj_qty', 0, q1).sse, 0, 1, .01) //alpha optimized for q1, using non smoothed seasonal estimate
    alpha_q2 = golden_section_search(x => fit(x, 'st_deseasonalized_adj_qty', q1 + 1, q2).sse, 0, 1, .01)
    alpha_q3 = golden_section_search(x => fit(x, 'st_deseasonalized_adj_qty', q2 + 1, q3).sse, 0, 1, .01)
    alpha_q4 = golden_section_search(x => fit(x, 'st_deseasonalized_adj_qty', q3 + 1, last).sse, 0, 1, .01)
    
    //fit updates data in place
    fit(alpha_h2, 'deseasonalized_adj_qty', 0, 0, 'fcst_deseasonalized_qty', 0, h1) //using alpha optimized for h2 to predict h1
    fit(alpha_h1, 'deseasonalized_adj_qty', 0, 0, 'fcst_deseasonalized_qty', h1 + 1, last - 1) //using alpha optimized for h1 to predict h2
    fit(alpha, 'deseasonalized_adj_qty', 0, 0, 'fcst_deseasonalized_qty', last, last) //using alpha optimized for all data
    fit(alpha_q4, 'st_deseasonalized_adj_qty', 0, 0, 'st_fcst_deseasonalized_qty', 0, last) //use alpha optimized for q4 to predict q1 and last - filling in like this and overwriting below to avoid an extra pass over data
    fit(alpha_q1, 'st_deseasonalized_adj_qty', 0, 0, 'st_fcst_deseasonalized_qty', q1 + 1, q2) //using alpha optimized for q1 to predict q2
    fit(alpha_q2, 'st_deseasonalized_adj_qty', 0, 0, 'st_fcst_deseasonalized_qty', q2 + 1, q3) //using alpha optimized for q2 to predict q3
    fit(alpha_q3, 'st_deseasonalized_adj_qty', 0, 0, 'st_fcst_deseasonalized_qty', q3 + 1, last - 1) //using alpha optimized for q3 to predict q4
    
    return {data: data, alpha: alpha, alpha_h1: alpha_h1, alpha_h2: alpha_h2, alpha_q1: alpha_q1, alpha_q2: alpha_q2, alpha_q3: alpha_q3, alpha_q4: alpha_q4}
    """;

    CREATE OR REPLACE TABLE `genericDataset.BQML_EXP_SMOOTH_OUTPUT` genericTableOptionList AS (
    WITH TBL1 AS (
    SELECT KEY, EXP_SMOOTH(ARRAY_AGG(STRUCT(DT, DESEASONALIZED_ADJ_QTY, ST_DESEASONALIZED_ADJ_QTY) ORDER BY DT)) AS EXP_SMOOTH_OUTPUT
    FROM `genericDataset.BQML_EXP_SMOOTH_DATA`
    WHERE FCST_CRT_DT_FLG
    GROUP BY 1
    )
    SELECT A.KEY, B.*, A.EXP_SMOOTH_OUTPUT.* EXCEPT(DATA)
    FROM TBL1 A
    CROSS JOIN UNNEST(A.EXP_SMOOTH_OUTPUT.DATA) B
    );
    ''',replaceDict, 'createForecast')
    with out:
        print(datetime.datetime.now().strftime('%H:%M:%S'), 'Forecast 2 step 7/9 completed...')
        print(datetime.datetime.now().strftime('%H:%M:%S'), 'Forecast 3 step 1/3 completed...')

    exec_sql('''
    CREATE OR REPLACE TABLE `genericDataset.BQML_FCST` genericTableOptionList AS (
    SELECT A.KEY
        --, A.DT AS FCST_CRT_DT
        , A.NEXT_DT AS FCST_CRT_DT
        , FCST_DT
        , LAG + 1 AS LAG --INDEX STARTS AT 0
        , C.QTY
        , C.ADJ_QTY
        
        --LONG TERM FORECAST
        , C.DESEASONALIZED_ADJ_QTY
        , D.FCST_DESEASONALIZED_QTY
        , C.SMOOTHED_SEASONAL_QTY
        , D.FCST_DESEASONALIZED_QTY + C.SMOOTHED_SEASONAL_QTY AS FCST_QTY
        
        --SHORT TERM FORECAST
        , C.ST_DESEASONALIZED_ADJ_QTY
        , D.ST_FCST_DESEASONALIZED_QTY
        , C.SEASONAL_QTY
        , D.ST_FCST_DESEASONALIZED_QTY + C.SEASONAL_QTY AS ST_FCST_QTY
    FROM `genericDataset.BQML_EXP_SMOOTH_DATA` A
    CROSS JOIN UNNEST(A.FCST_DTS) FCST_DT WITH OFFSET LAG
    INNER JOIN `genericDataset.BQML_EXP_SMOOTH_DATA` C
        ON A.KEY = C.KEY AND FCST_DT = C.DT
    INNER JOIN `genericDataset.BQML_EXP_SMOOTH_OUTPUT` D
        ON A.KEY = D.KEY AND A.DT = D.DT
    WHERE A.FCST_CRT_DT_FLG
    );
    ''',replaceDict, 'createForecast')
    with out:
        print(datetime.datetime.now().strftime('%H:%M:%S'), 'Forecast 2 step 8/9 completed...')
        print(datetime.datetime.now().strftime('%H:%M:%S'), 'Forecast 3 step 2/3 completed...')

    exec_sql('''
    UPDATE `genericDataset.genericOutputTable` A
    SET A.FORECAST2_genericForecastValue = coalesce( forecastTransform1 B.FCST_QTY forecastTransform2 , 0)
        , A.FORECAST3_genericForecastValue = coalesce( forecastTransform1 B.ST_FCST_QTY forecastTransform2, 0)
    FROM `genericDataset.BQML_FCST` B
    WHERE A.SNPSHT_genericTimePeriod = B.FCST_CRT_DT
        AND A.FCST_genericTimePeriod = B.FCST_DT
        AND A.AGGLEVEL = B.KEY;
    ''',replaceDict, 'createForecast'
    ,altSQL='''
    UPDATE `genericDataset.genericOutputTable` A
    SET A.FORECAST2_genericForecastValue = coalesce( forecastTransform1 B.FCST_QTY forecastTransform2 , 0)
        , A.FORECAST3_genericForecastValue = coalesce( forecastTransform1 B.ST_FCST_QTY forecastTransform2, 0)
    FROM `genericDataset.BQML_FCST` B
    WHERE A.SNPSHT_genericTimePeriod = B.FCST_CRT_DT
        AND A.FCST_genericTimePeriod = B.FCST_DT
        AND A.AGGLEVEL = B.KEY
        AND B.FCST_CRT_DT = date_trunc(current_date(), genericFrequency);--update for current day/week only
    '''
        )

    with out:
        print(datetime.datetime.now().strftime('%H:%M:%S'), 'Forecast 2 step 9/9 completed, forecast 2 created successfully.')
        print(datetime.datetime.now().strftime('%H:%M:%S'), 'Forecast 3 step 3/3 completed, forecast 3 created successfully.')
    #display(Javascript("execCellsByTag('Forecast3')"))

except:
    with out:
        print(datetime.datetime.now().strftime('%H:%M:%S'), 'Error: Invalid input detected while generating forecast 2. Please verify inputs and try again.')

In [ ]:
#selection logic tables

try:
    exec_sql(
    '''create or replace table `genericDataset.forecast_selection` as (
    SELECT snpsht_genericTimePeriod, FCST_genericTimePeriod
    , aggLevel
    ,actual_genericForecastValue as ACTUAL_genericForecastValue
    , greatest(coalesce(forecast1_genericForecastValue,forecast1_genericForecastValue, 0),0) as Forecast1_genericForecastValue
    , coalesce(safe_divide(abs(coalesce(forecast1_genericForecastValue,forecast1_genericForecastValue, 0)-Actual_genericForecastValue),(Actual_genericForecastValue)),1.0) as forecast1_MAPE
    ,coalesce(Forecast2_genericForecastValue,0) as Forecast2_genericForecastValue
    ,coalesce(safe_divide(abs(coalesce(Forecast2_genericForecastValue, forecast2_genericForecastValue,0)-Actual_genericForecastValue),(Actual_genericForecastValue)),1.0) as forecast2_MAPE
    ,greatest(forecast3_genericForecastValue,0) as forecast3_genericForecastValue
    ,coalesce(safe_divide(abs(coalesce(forecast3_genericForecastValue,0)-Actual_genericForecastValue),(Actual_genericForecastValue)),1.0) as forecast3_MAPE
    FROM `genericDataset.genericOutputTable`
    where
    (fcst_genericTimePeriod = snpsht_genericTimePeriod and
    fcst_genericTimePeriod in (SELECT distinct genericTimePeriod From `pr-edw-views-thd.SHARED.CAL_PRD_HIER_FD` where 
    (select min(fcst_genericTimePeriod) from `genericDataset.genericOutputTable`) <= CAL_DT
    and date_trunc(current_date(), genericFrequency) >= CAL_DT))
    );
    ''',ensembleReplaceDict, 'ensemble')
    
    with selectionOut:
        print(datetime.datetime.now().strftime('%H:%M:%S'), 'Selection logic input table 1/2 completed...')

    exec_sql(
        '''create or replace table `genericDataset.forecast_stats` genericTableOptionList as (
        WITH TBL2A AS (
        SELECT A.FCST_genericTimePeriod, B.EVAL_genericTimePeriod
        FROM
        (
        SELECT RANK() OVER (ORDER BY fcst_genericTimePeriod) TIME_RANK, fcst_genericTimePeriod from
        (SELECT DISTINCT genericTimePeriod FCST_genericTimePeriod FROM `pr-edw-views-thd.SHARED.CAL_PRD_HIER_FD`
        where cal_dt between (select min(genericTimePeriod) from `genericDataset.forecast_data`) and CAST('forecastPeriodEnd' as date))
        ORDER BY 1
        ) A
        INNER JOIN
        (
        SELECT RANK() OVER (ORDER BY eval_genericTimePeriod) TIME_RANK, eval_genericTimePeriod from
        (SELECT DISTINCT genericTimePeriod EVAL_genericTimePeriod FROM `pr-edw-views-thd.SHARED.CAL_PRD_HIER_FD`
        where cal_dt between (select min(genericTimePeriod) from `genericDataset.forecast_data`) and date_trunc(current_date(), genericFrequency))
        ORDER BY 1
        ) B
        ON A.TIME_RANK - B.TIME_RANK BETWEEN 1 AND 6--edit time rank to change lag for rolling MAPE
        --WHERE
        --genericTimePeriod in (SELECT distinct genericTimePeriod From `pr-edw-views-thd.SHARED.CAL_PRD_HIER_FD` where 
        --CAST(cast('forecastPeriodStart' as string) as date) <= CAL_DT
        --and CAST(cast('forecastPeriodEnd' as string) as date) >= CAL_DT)
        --and A.TIME_RANK > B.TIME_RANK
        --and FLOOR((A.TIME_RANK - B.TIME_RANK) / 52) = 0
        ORDER BY 1, 2
        )
        ,TBL2A_short_period AS (
        SELECT A.FCST_genericTimePeriod, B.EVAL_genericTimePeriod
        FROM
        (
        SELECT RANK() OVER (ORDER BY fcst_genericTimePeriod) TIME_RANK, fcst_genericTimePeriod from
        (SELECT DISTINCT genericTimePeriod FCST_genericTimePeriod FROM `pr-edw-views-thd.SHARED.CAL_PRD_HIER_FD`
        where cal_dt between (select min(genericTimePeriod) from `genericDataset.forecast_data`) and date_trunc(current_date(), genericFrequency))
        ORDER BY 1
        ) A
        INNER JOIN
        (
        SELECT RANK() OVER (ORDER BY eval_genericTimePeriod) TIME_RANK, eval_genericTimePeriod from
        (SELECT DISTINCT genericTimePeriod EVAL_genericTimePeriod FROM `pr-edw-views-thd.SHARED.CAL_PRD_HIER_FD`
        where cal_dt between (select min(genericTimePeriod) from `genericDataset.forecast_data`) and date_trunc(current_date(), genericFrequency))
        ORDER BY 1
        ) B
        ON A.TIME_RANK - B.TIME_RANK BETWEEN 1 AND 2--edit time rank to change lag for rolling MAPE
        ORDER BY 1, 2
        ),
        TBL2A_long_period AS (
        SELECT A.FCST_genericTimePeriod, B.EVAL_genericTimePeriod
        FROM
        (
        SELECT RANK() OVER (ORDER BY fcst_genericTimePeriod) TIME_RANK, fcst_genericTimePeriod from
        (SELECT DISTINCT genericTimePeriod FCST_genericTimePeriod FROM `pr-edw-views-thd.SHARED.CAL_PRD_HIER_FD`
        where cal_dt between (select min(genericTimePeriod) from `genericDataset.forecast_data`) and date_trunc(current_date(), genericFrequency))
        ORDER BY 1
        ) A
        INNER JOIN
        (
        SELECT RANK() OVER (ORDER BY eval_genericTimePeriod) TIME_RANK, eval_genericTimePeriod from
        (SELECT DISTINCT genericTimePeriod EVAL_genericTimePeriod FROM `pr-edw-views-thd.SHARED.CAL_PRD_HIER_FD`
        where cal_dt between (select min(genericTimePeriod) from `genericDataset.forecast_data`) and date_trunc(current_date(), genericFrequency))
        ORDER BY 1
        ) B
        ON A.TIME_RANK - B.TIME_RANK BETWEEN 1 AND 52--edit time rank to change lag for rolling MAPE
        ORDER BY 1, 2
        ),
        avg_mape as (
        select fcst_genericTimePeriod, aggLevel
        , avg(forecast1_mape) as avg_forecast1_mape, stddev(forecast1_mape) as stddev_forecast1_mape
        , avg(forecast2_mape) as avg_forecast2_mape, stddev(forecast2_mape) as stddev_forecast2_mape
        , avg(forecast3_mape) as avg_forecast3_mape, stddev(forecast3_mape) as stddev_forecast3_mape
        from
        (select a.*, b.* except (FCST_genericTimePeriod) from TBL2A a
        inner join `genericDataset.forecast_selection` b
        on a.eval_genericTimePeriod = b.FCST_genericTimePeriod) c
        where aggLevel is not null
        group by 1,2
        union all
        select * from
        (select min(fcst_genericTimePeriod) from `genericDataset.forecast_selection`)
        cross join
        (select distinct aggLevel, null, null, null, null, null, null--, null, null
        from `genericDataset.forecast_selection`)
        ),
        avg_mape_short as (
        select fcst_genericTimePeriod, aggLevel
        , avg(forecast1_mape) as avg_forecast1_mape_short, stddev(forecast1_mape) as stddev_forecast1_mape_short
        , avg(forecast2_mape) as avg_forecast2_mape_short, stddev(forecast2_mape) as stddev_forecast2_mape_short
        , avg(forecast3_mape) as avg_forecast3_mape_short, stddev(forecast3_mape) as stddev_forecast3_mape_short
        from
        (select a.*, b.* except (FCST_genericTimePeriod) from TBL2A_short_period a
        inner join `genericDataset.forecast_selection` b
        on a.eval_genericTimePeriod = b.FCST_genericTimePeriod) c
        group by 1,2
        union all
        select * from
        (select min(fcst_genericTimePeriod) from `genericDataset.forecast_selection`)
        cross join
        (select distinct aggLevel, null, null, null, null, null, null--, null, null
        from `genericDataset.forecast_selection`)
        ),
        avg_mape_long as (
        select fcst_genericTimePeriod, aggLevel
        , avg(forecast1_mape) as overall_avg_forecast1_mape, stddev(forecast1_mape) as overall_stddev_forecast1_mape
        , avg(forecast2_mape) as overall_avg_forecast2_mape, stddev(forecast2_mape) as overall_stddev_forecast2_mape
        , avg(forecast3_mape) as overall_avg_forecast3_mape, stddev(forecast3_mape) as overall_stddev_forecast3_mape
        from
        (select a.*, b.* except (FCST_genericTimePeriod) from TBL2A_long_period a
        inner join `genericDataset.forecast_selection` b
        on a.eval_genericTimePeriod = b.FCST_genericTimePeriod) c
        where aggLevel is not null
        group by 1,2
        union all
        select * from
        (select min(fcst_genericTimePeriod) from `genericDataset.forecast_selection`)
        cross join
        (select distinct aggLevel, null, null, null, null, null, null--, null, null
        from `genericDataset.forecast_selection`)
        ),
        forecast_stats AS
        (
        select a.fcst_genericTimePeriod, a.aggLevel
        ,d.actual_genericForecastValue, forecast1_genericForecastValue
        ,forecast2_genericForecastValue, forecast3_genericForecastValue
        ,forecast1_mape
        ,forecast2_mape
        ,forecast3_mape
        ,case
        when forecast1_mape > forecast2_mape and forecast3_mape > forecast2_mape
        and forecast2_mape is not null
        then "forecast2"
        when forecast2_mape > forecast3_mape and forecast1_mape > forecast3_mape
        and forecast3_mape is not null
        then "forecast3"
        when forecast2_mape > forecast1_mape and forecast3_mape > forecast1_mape
        then "forecast1"
        else "Default - FORECAST2" end as best_forecast
        ,least(avg_forecast1_mape/*, avg_fin_mape, avg_is_mape, avg_nm_mape*/, avg_forecast2_mape, avg_forecast3_mape) as least_avg_mape
        ,least(avg_forecast1_mape_short/*, avg_fin_mape_short, avg_is_mape_short, avg_nm_mape_short*/, avg_forecast2_mape_short, avg_forecast3_mape_short) as least_avg_mape_short
        ,least(overall_avg_forecast1_mape, overall_avg_forecast2_mape/*, overall_avg_is_mape, overall_avg_nm_mape*/, overall_avg_forecast3_mape) as overall_least_avg_mape
        ,least(stddev_forecast1_mape, stddev_forecast2_mape/*, stddev_is_mape, stddev_nm_mape*/, stddev_forecast3_mape) as least_stddev_mape
        ,least(overall_stddev_forecast1_mape,overall_stddev_forecast2_mape/*, overall_stddev_is_mape, overall_stddev_nm_mape*/, overall_stddev_forecast3_mape) as overall_least_stddev_mape
        ,coalesce(avg_forecast1_mape,1) as avg_forecast1_mape,coalesce(overall_avg_forecast1_mape,1) as overall_avg_forecast1_mape,coalesce(avg_forecast1_mape_short,1) as avg_forecast1_mape_short
        ,coalesce(stddev_forecast1_mape,0) as stddev_forecast1_mape,coalesce(overall_stddev_forecast1_mape,0) as overall_stddev_forecast1_mape
        ,coalesce(avg_forecast2_mape,1) as avg_forecast2_mape,coalesce(overall_avg_forecast2_mape,1) as overall_avg_forecast2_mape,coalesce(avg_forecast2_mape_short,1) as avg_forecast2_mape_short
        ,coalesce(stddev_forecast2_mape,0) as stddev_forecast2_mape,coalesce(overall_stddev_forecast2_mape,0) as overall_stddev_forecast2_mape
        ,coalesce(avg_forecast3_mape,1) as avg_forecast3_mape,coalesce(overall_avg_forecast3_mape,1) as overall_avg_forecast3_mape,coalesce(avg_forecast3_mape_short,1) as avg_forecast3_mape_short
        ,coalesce(stddev_forecast3_mape,0) as stddev_forecast3_mape,coalesce(overall_stddev_forecast3_mape,0) as overall_stddev_forecast3_mape
        , e.time_rank
        from avg_mape a
        inner join avg_mape_long b
        on a.fcst_genericTimePeriod = b.fcst_genericTimePeriod
        and a.aggLevel = b.aggLevel
        inner join avg_mape_short c
        on a.fcst_genericTimePeriod = c.fcst_genericTimePeriod
        and a.aggLevel = c.aggLevel
        inner join `genericDataset.forecast_selection` d
        on a.fcst_genericTimePeriod = d.fcst_genericTimePeriod
        and a.aggLevel = d.aggLevel
        inner join
        (SELECT DISTINCT RANK() OVER (ORDER BY FCST_genericTimePeriod) TIME_RANK, fcst_genericTimePeriod from (select distinct FCST_genericTimePeriod FROM `genericDataset.forecast_selection`)
        ORDER BY 1) e
        on a.fcst_genericTimePeriod = e.fcst_genericTimePeriod
        )


        select a.best_forecast as prev_best_forecast, b.*
        , b.avg_forecast1_mape - b.least_avg_mape as least_avg_forecast1_mape
        , b.avg_forecast2_mape - b.least_avg_mape as least_avg_forecast2_mape
        , b.avg_forecast3_mape - b.least_avg_mape as least_avg_forecast3_mape
        , b.avg_forecast1_mape_short - b.least_avg_mape_short as least_avg_forecast1_mape_short
        , b.avg_forecast2_mape_short - b.least_avg_mape_short as least_avg_forecast2_mape_short
        , b.avg_forecast3_mape_short - b.least_avg_mape_short as least_avg_forecast3_mape_short
        , b.overall_avg_forecast1_mape - b.overall_least_avg_mape as overall_least_avg_forecast1_mape
        , b.overall_avg_forecast2_mape - b.overall_least_avg_mape as overall_least_avg_forecast2_mape
        , b.overall_avg_forecast3_mape - b.overall_least_avg_mape as overall_least_avg_forecast3_mape
        , b.stddev_forecast1_mape - b.least_stddev_mape as least_stddev_forecast1_mape
        , b.stddev_forecast2_mape - b.least_stddev_mape as least_stddev_forecast2_mape
        , b.stddev_forecast3_mape - b.least_stddev_mape as least_stddev_forecast3_mape
        , b.overall_stddev_forecast1_mape - b.overall_least_stddev_mape as overall_least_stddev_forecast1_mape
        , b.overall_stddev_forecast2_mape - b.overall_least_stddev_mape as overall_least_stddev_forecast2_mape
        , b.overall_stddev_forecast3_mape - b.overall_least_stddev_mape as overall_least_stddev_forecast3_mape
        from forecast_stats a
        right join forecast_stats b
        on b.time_rank - a.time_rank = 1
        and a.aggLevel = b.aggLevel
        );
    ''',ensembleReplaceDict, 'ensemble')

    
    with selectionOut:
        print(datetime.datetime.now().strftime('%H:%M:%S'), 'Selection logic input table 2/2 completed...')
    
except:
    with selectionOut:
        print(datetime.datetime.now().strftime('%H:%M:%S'), 'Error: Invalid input detected while generating selection input tables. Please verify inputs and try again.')


In [ ]:
#rules based selection

try:
    exec_sql(
    '''
    create or replace table `genericDataset.one_selection` genericTableOptionList as (

    with pre_one_selection as (
    select
    coalesce(
    case
    when 
    least(least_avg_forecast1_mape, least_avg_forecast2_mape, least_avg_forecast3_mape) = least_avg_forecast2_mape
    and (least_avg_forecast1_mape = least_avg_forecast2_mape or least_avg_forecast2_mape = least_avg_forecast3_mape)
    then forecast2_genericForecastValue
    when
    least(least_avg_forecast1_mape, least_avg_forecast2_mape, least_avg_forecast3_mape) = least_avg_forecast3_mape
    and (least_avg_forecast1_mape = least_avg_forecast3_mape)
    then coalesce(forecast1_genericForecastValue, 0)
    when least(least_avg_forecast1_mape, least_avg_forecast2_mape, least_avg_forecast3_mape)
    = least_avg_forecast2_mape then forecast2_genericForecastValue 
    when least(least_avg_forecast1_mape, least_avg_forecast2_mape, least_avg_forecast3_mape)
    = least_avg_forecast3_mape then coalesce(forecast3_genericForecastValue,0)
    when least(least_avg_forecast1_mape, least_avg_forecast2_mape, least_avg_forecast3_mape)
    = least_avg_forecast1_mape then coalesce(forecast1_genericForecastValue,0)
    end,0)
    as one_genericForecastValue
    ,case
    when 
    least(least_avg_forecast1_mape, least_avg_forecast2_mape, least_avg_forecast3_mape) = least_avg_forecast2_mape
    and (least_avg_forecast1_mape = least_avg_forecast2_mape or least_avg_forecast2_mape = least_avg_forecast3_mape)
    then 'Default - Forecast2' 
    when 
    least(least_avg_forecast1_mape, least_avg_forecast2_mape, least_avg_forecast3_mape) = least_avg_forecast3_mape
    and (least_avg_forecast1_mape = least_avg_forecast3_mape)
    then 'Default - Forecast1'
    when least(least_avg_forecast1_mape, least_avg_forecast2_mape, least_avg_forecast3_mape)
    = least_avg_forecast2_mape then 'Forecast2' 
    when least(least_avg_forecast1_mape, least_avg_forecast2_mape, least_avg_forecast3_mape)
    = least_avg_forecast3_mape then 'Forecast3'
    when least(least_avg_forecast1_mape, least_avg_forecast2_mape, least_avg_forecast3_mape)
    = least_avg_forecast1_mape then 'Forecast1'
    end as one_selection
    ,actual_genericForecastValue
    ,forecast1_genericForecastValue
    ,forecast2_genericForecastValue
    ,forecast3_genericForecastValue
    ,fcst_genericTimePeriod
    ,aggLevel
    , least_avg_forecast1_mape, least_avg_forecast2_mape, least_avg_forecast3_mape
    , case
    when least(abs(forecast1_genericForecastValue - coalesce(actual_genericForecastValue,0))
    ,abs(forecast3_genericForecastValue - coalesce(actual_genericForecastValue,0))
    ,abs(forecast2_genericForecastValue - coalesce(actual_genericForecastValue,0))) = abs(forecast2_genericForecastValue - coalesce(actual_genericForecastValue,0)) then forecast2_genericForecastValue
    when least(abs(forecast1_genericForecastValue - coalesce(actual_genericForecastValue,0))
    ,abs(forecast3_genericForecastValue - coalesce(actual_genericForecastValue,0))
    ,abs(forecast2_genericForecastValue - coalesce(actual_genericForecastValue,0))) = abs(forecast1_genericForecastValue - coalesce(actual_genericForecastValue,0)) then forecast1_genericForecastValue
    when least(abs(forecast1_genericForecastValue - coalesce(actual_genericForecastValue,0))
    ,abs(forecast3_genericForecastValue - coalesce(actual_genericForecastValue,0))
    ,abs(forecast2_genericForecastValue - coalesce(actual_genericForecastValue,0))) = abs(forecast3_genericForecastValue - coalesce(actual_genericForecastValue,0)) then forecast3_genericForecastValue
    end as best_genericForecastValue
    from
    `genericDataset.forecast_stats`
    where current_date() >= fcst_genericTimePeriod
    ),
    recent_best as (
    select aggLevel, one_selection, least_avg_forecast1_mape, least_avg_forecast2_mape, least_avg_forecast3_mape from pre_one_selection
    where fcst_genericTimePeriod = (SELECT max(genericTimePeriod) FROM `pr-edw-views-thd.SHARED.CAL_PRD_HIER_FD` where 
    current_date() >= FSCL_WK_BGN_DT
    and current_date() <= FSCL_WK_END_DT)
    )



    --selection for 6 weeks from current week
    select * from pre_one_selection
    union all
    select
    coalesce(case 
    when 
    least(least_avg_forecast1_mape, least_avg_forecast2_mape, least_avg_forecast3_mape) = least_avg_forecast2_mape
    and (least_avg_forecast1_mape = least_avg_forecast2_mape or least_avg_forecast2_mape = least_avg_forecast3_mape)
    then a.forecast2_genericForecastValue
    when 
    least(least_avg_forecast1_mape, least_avg_forecast2_mape, least_avg_forecast3_mape) = least_avg_forecast3_mape
    and (least_avg_forecast1_mape = least_avg_forecast3_mape)
    then coalesce(b.forecast3_genericForecastValue, 0)
    when least(least_avg_forecast1_mape, least_avg_forecast2_mape, least_avg_forecast3_mape)
    = least_avg_forecast2_mape then a.forecast2_genericForecastValue 
    when least(least_avg_forecast1_mape, least_avg_forecast2_mape, least_avg_forecast3_mape)
    = least_avg_forecast3_mape then coalesce(b.forecast3_genericForecastValue,0)
    when least(least_avg_forecast1_mape, least_avg_forecast2_mape, least_avg_forecast3_mape)
    = least_avg_forecast1_mape then coalesce(a.forecast1_genericForecastValue,0)
    end,0) as one_genericForecastValue
    ,c.one_selection
    ,a.actual_genericForecastValue
    ,coalesce(a.forecast1_genericForecastValue,0) as forecast1_genericForecastValue
    ,coalesce(a.forecast2_genericForecastValue,0) as forecast2_genericForecastValue
    ,coalesce(b.forecast3_genericForecastValue,0) as forecast3_genericForecastValue
    ,a.fcst_genericTimePeriod
    ,a.aggLevel
    ,null,null,null,null
    from
    `genericDataset.genericOutputTable` a
    ,`genericDataset.genericOutputTable` b
    ,recent_best c
    where a.fcst_genericTimePeriod >= (SELECT max(genericTimePeriod) FROM `pr-edw-views-thd.SHARED.CAL_PRD_HIER_FD` where 
    current_date() >= FSCL_WK_BGN_DT
    and current_date() <= FSCL_WK_END_DT)
    and a.snpsht_genericTimePeriod = (SELECT max(genericTimePeriod) FROM `pr-edw-views-thd.SHARED.CAL_PRD_HIER_FD` where 
    current_date() >= FSCL_WK_BGN_DT
    and current_date() <= FSCL_WK_END_DT)
    and a.aggLevel = b.aggLevel
    and a.aggLevel = c.aggLevel
    and a.fcst_genericTimePeriod = b.snpsht_genericTimePeriod
    and b.fcst_genericTimePeriod = b.snpsht_genericTimePeriod
    );
    ''',ensembleReplaceDict, 'ensemble')

    
    exec_sql(
    ''' update `genericDataset.genericOutputTable` a
        set a.one_forecast_genericForecastValue  =
        case
        when 
        least(least_avg_forecast1_mape, least_avg_forecast2_mape, least_avg_forecast3_mape) = least_avg_forecast2_mape
        and (least_avg_forecast1_mape = least_avg_forecast2_mape or least_avg_forecast2_mape = least_avg_forecast3_mape)
        then a.forecast2_genericForecastValue
        when 
        least(least_avg_forecast1_mape, least_avg_forecast2_mape, least_avg_forecast3_mape) = least_avg_forecast3_mape
        and (least_avg_forecast1_mape = least_avg_forecast3_mape)
        then a.forecast1_genericForecastValue
        when least(least_avg_forecast1_mape, least_avg_forecast2_mape, least_avg_forecast3_mape)
        = least_avg_forecast2_mape then a.forecast2_genericForecastValue
        when least(least_avg_forecast1_mape, least_avg_forecast2_mape, least_avg_forecast3_mape)
        = least_avg_forecast3_mape then a.forecast3_genericForecastValue
        when least(least_avg_forecast1_mape, least_avg_forecast2_mape, least_avg_forecast3_mape)
        = least_avg_forecast1_mape then a.forecast1_genericForecastValue
        end
        FROM `genericDataset.one_selection` b
        where a.snpsht_genericTimePeriod = b.fcst_genericTimePeriod
        and a.aggLevel = b.aggLevel;
    ''',ensembleReplaceDict, 'ensemble'
    ,altSQL='''update `genericDataset.genericOutputTable` a
        set a.one_forecast_genericForecastValue  =
        case
        when
        least(least_avg_forecast1_mape, least_avg_forecast2_mape, least_avg_forecast3_mape) = least_avg_forecast2_mape
        and (least_avg_forecast1_mape = least_avg_forecast2_mape or least_avg_forecast2_mape = least_avg_forecast3_mape)
        then a.forecast2_genericForecastValue
        when 
        least(least_avg_forecast1_mape, least_avg_forecast2_mape, least_avg_forecast3_mape) = least_avg_forecast3_mape
        and (least_avg_forecast1_mape = least_avg_forecast3_mape)
        then a.forecast1_genericForecastValue
        when least(least_avg_forecast1_mape, least_avg_forecast2_mape, least_avg_forecast3_mape)
        = least_avg_forecast2_mape then a.forecast2_genericForecastValue
        when least(least_avg_forecast1_mape, least_avg_forecast2_mape, least_avg_forecast3_mape)
        = least_avg_forecast3_mape then a.forecast3_genericForecastValue
        when least(least_avg_forecast1_mape, least_avg_forecast2_mape, least_avg_forecast3_mape)
        = least_avg_forecast1_mape then a.forecast1_genericForecastValue
        end
        FROM `genericDataset.one_selection` b
        where a.snpsht_genericTimePeriod = b.fcst_genericTimePeriod
        and a.snpsht_genericTimePeriod = date_trunc(current_date(), genericFrequency)
        and a.aggLevel = b.aggLevel;'''
    )
    
    with selectionOut:
        print(datetime.datetime.now().strftime('%H:%M:%S'), genericReplace('Rules based selection output stored in `genericDataset.genericOutputTable`. Forecast process complete.',replaceDict))
except:
    with selectionOut:
        print(datetime.datetime.now().strftime('%H:%M:%S'), 'Error: Invalid input detected while generating rules based selection. Please verify inputs and try again.')


In [ ]:
#RL selection

try:
    exec_sql(
    '''CREATE OR REPLACE FUNCTION  `genericDataset.learn`(history ARRAY<STRUCT<fcst_genericTimePeriod DATE, ONE_MAPE FLOAT64
        , least_avg_forecast1_mape FLOAT64, least_avg_forecast2_mape FLOAT64, least_avg_forecast3_mape FLOAT64
        , forecast1_genericForecastValue FLOAT64, forecast2_genericForecastValue FLOAT64, forecast3_genericForecastValue FLOAT64, actual_genericForecastValue FLOAT64
        ,forecast1_MAPE FLOAT64, forecast2_MAPE FLOAT64, forecast3_MAPE FLOAT64, Min_MAPE FLOAT64, Max_MAPE FLOAT64>>)
        RETURNS ARRAY<STRUCT<fcst_genericTimePeriod DATE, ONE_QTY FLOAT64, forecast1_QTY FLOAT64, forecast2_QTY FLOAT64, forecast3_QTY FLOAT64, ACTUAL_QTY FLOAT64
        , ONE_MAPE FLOAT64, forecast1_MAPE FLOAT64, forecast2_MAPE FLOAT64, forecast3_MAPE FLOAT64, FORECAST_CHOSEN STRING, reward FLOAT64>> LANGUAGE js
    OPTIONS (library=["JSCloudPath/rl.js"]) AS """
            var states = []//array of states
            var actualMAPE = []//array of periodly MAPE values
            var qty = []//array of each forecasts periodly qty
            var genericTimePeriod = []//array of fscl periods
            var data = history.map(function(obj) {
              return Object.keys(obj).map(function(key) {
                return obj[key]
              })
            })
            var i = 0
            while(i < data.length) {
                states.push(data[i].slice(1,5))
                actualMAPE.push(data[i].slice(9))
                qty.push(data[i].slice(5,9))
                genericTimePeriod.push(data[i][0])
                i++
            }

            var env = {}
            env.getNumStates = function() { return 4 }//One, FORECAST1, FORECAST2, FORECAST3 MAPE values
            env.getMaxNumActions = function() { return 3 }//action is forecast chosen for next period
            //0 = FORECAST1, 1 = FORECAST2, 2 = FORECAST3

            // create the DQN agent
            //need to take experience size 
            var numPeriods = genericTimePeriod.length
            var spec = { alpha: 0.1, experience_size: numPeriods * .2, experience_add_every: 1, epsilon: 1, gamma: 0, learning_steps_per_iteration: 10 } // see full options on DQN page
            //test different options for alpha and epsilon
            //Gamma = 1 = max greediness, does not consider potential applications from future states
            agent = new RL.DQNAgent(env, spec)
            var selectedMAPE = []
            var forecast = []
            var period = 0
            while(period < numPeriods)
            {
                agent.epsilon = 1 - (period)/(numPeriods * .8)

                if (period > numPeriods * .8) {//stop exploring before last year
                    agent.epsilon = 0
                }
                var state = states[period]//set state
                if (period > 0) {
                    //console.log(state[period][0])
                    state[0] = selectedMAPE[period - 1]//update ONE MAPE value of state to be MAPE of forecast selected LAST PERIOD
                } else {
                    state[0] = 1//what should default be?
                }
                var action = agent.act(state) // s is an array of length NumStates ... execute action in environment and get the reward

                //states[period][0] = actualMAPE[period][action]//update state with least MAPE calculation mape for next period from chosen action
                selectedMAPE.push(actualMAPE[period][action])
                //update s1 which is updated based on action in agent.act
                var selection = ["forecast1","forecast2","forecast3"]
                var reward = 1 - ((actualMAPE[period][action] - actualMAPE[period][3])/(actualMAPE[period][4]-actualMAPE[period][3]))//1 - (selected MAPE - lowest MAPE) / (highest MAPE - lowest MAPE)
                if (actualMAPE[period][0] == actualMAPE[period][1] && actualMAPE[period][0] == actualMAPE[period][2]) {
                    reward = 0
                    //reward = 100 * actualMAPE[period][action]
                }
                else if (reward > .5) {
                    reward *= 100 * (actualMAPE[period][4]-actualMAPE[period][3])//scale by difference between max and min
                    reward = Math.min(reward, 100)
                } else {
                    reward = -100 * (1-reward) * (actualMAPE[period][4]-actualMAPE[period][3])
                    reward = Math.max(reward, -100)
                }
                agent.learn(reward) // the agent improves its Q,policy,model, etc. reward is a float, also randomly samples an experience from memory

                //generate output
                if (!qty[period][3]) {//don't calculate MAPE if actuals are null
                    forecast.push({fcst_genericTimePeriod:genericTimePeriod[period],ONE_QTY:qty[period][action], forecast1_QTY:qty[period][0], forecast2_QTY:qty[period][1], forecast3_QTY:qty[period][2]
                        ,ACTUAL_QTY:qty[period][3]
                        ,FORECAST_CHOSEN:selection[action]
                        ,reward:reward}
                    )
                } else {
                    forecast.push({fcst_genericTimePeriod:genericTimePeriod[period],ONE_QTY:qty[period][action], forecast1_QTY:qty[period][0], forecast2_QTY:qty[period][1], forecast3_QTY:qty[period][2]
                        ,ACTUAL_QTY:qty[period][3]
                        ,ONE_MAPE:(Math.abs(qty[period][action] - qty[period][3])/qty[period][3]).toFixed(4)
                        ,forecast1_MAPE:(Math.abs(qty[period][0] - qty[period][3])/qty[period][3]).toFixed(4)
                        ,forecast2_MAPE:(Math.abs(qty[period][1] - qty[period][3])/qty[period][3]).toFixed(4)
                        ,forecast3_MAPE:(Math.abs(qty[period][2] - qty[period][3])/qty[period][3]).toFixed(4)
                        ,FORECAST_CHOSEN:selection[action]
                        ,reward:reward}
                    )
                }

                period++
          }
          return forecast



    """;
    ''',ensembleReplaceDict, 'ensemble')



    # call persistent UDF reinforcement learning model to select the best forecast each period
    # state: 6 wk rolling error for each of the three forecasts
    # action: pick one of the three forecasts
    # reward: positive value for below average error selection, negative value for above average error selection
    exec_sql(
    '''
    create or replace table `genericDataset.RL_output` as (
    select aggLevel, b.* from
    (select aggLevel, `genericDataset.learn`(ARRAY_AGG(STRUCT(fcst_genericTimePeriod, 0.0 as ONE_MAPE, least_avg_forecast1_MAPE, least_avg_forecast2_MAPE
    ,least_avg_forecast3_MAPE
    ,forecast1_genericForecastValue, forecast2_genericForecastValue, forecast3_genericForecastValue, cast( actual_genericForecastValue as float64)
    ,forecast1_mape, forecast2_mape, forecast3_mape
    ,least(forecast1_mape, forecast2_mape,forecast3_mape) as min_mape
    ,greatest(forecast1_mape, forecast2_mape, forecast3_mape) as max_mape) order by fcst_genericTimePeriod)) as RL_OUTPUT
    FROM `genericDataset.forecast_stats`
    where fcst_genericTimePeriod <= current_date()
    
    group by 1) a
    --having count(*) = (select count(distinct fcst_genericTimePeriod) from `genericDataset.forecast_stats` where fcst_genericTimePeriod <= current_date())) a
    ,unnest(RL_OUTPUT) b
    );
    ''',ensembleReplaceDict, 'ensemble'
    ,altSQL=
    '''create or replace table `genericDataset.RL_output` genericTableOptionList as (
    with tbl1 as (
    select aggLevel, b.* from
    (select aggLevel, `genericDataset.learn`(ARRAY_AGG(STRUCT(fcst_genericTimePeriod, 0.0 as ONE_MAPE, least_avg_forecast1_MAPE, least_avg_forecast2_MAPE
    ,least_avg_forecast3_MAPE
    ,forecast1_genericForecastValue, forecast2_genericForecastValue, forecast3_genericForecastValue, cast( actual_genericForecastValue as float64)
    ,forecast1_mape, forecast2_mape, forecast3_mape
    ,least(forecast1_mape, forecast2_mape,forecast3_mape) as min_mape
    ,greatest(forecast1_mape, forecast2_mape, forecast3_mape) as max_mape) order by fcst_genericTimePeriod)) as RL_OUTPUT
    FROM `genericDataset.forecast_stats`
    where fcst_genericTimePeriod <= current_date()
    
    group by 1) a
    --having count(*) = (select count(distinct fcst_genericTimePeriod) from `genericDataset.forecast_stats` where fcst_genericTimePeriod <= current_date())) a
    ,unnest(RL_OUTPUT) b
    )
    
    select * from `genericDataset.RL_output`
    union distinct
    select * from tbl1
    where fcst_genericTimePeriod >= (select max(fcst_genericTimePeriod) from `genericDataset.RL_output`)
    and fcst_genericTimePeriod <= current_date()
    
    );'''
    
    )

    exec_sql(
    ''' update `genericDataset.genericOutputTable` a
        set a.one_forecast_genericForecastValue  =
        case
        when b.forecast_chosen = 'forecast1'
        then b.forecast1_genericForecastValue
        when b.forecast_chosen = 'forecast2'
        then b.forecast2_genericForecastValue
        when b.forecast_chosen = 'forecast3'
        then b.forecast3_genericForecastValue
        end
        FROM `genericDataset.RL_output` b
        where a.snpsht_genericTimePeriod = b.fcst_genericTimePeriod
        and a.aggLevel = b.aggLevel;
    ''',ensembleReplaceDict, 'ensemble'
    ,altSQL = ''' update `genericDataset.genericOutputTable` a
        set a.one_forecast_genericForecastValue  =
        case
        when b.forecast_chosen = 'forecast1'
        then b.forecast1_genericForecastValue
        when b.forecast_chosen = 'forecast2'
        then b.forecast2_genericForecastValue
        when b.forecast_chosen = 'forecast3'
        then b.forecast3_genericForecastValue
        end
        FROM `genericDataset.RL_output` b
        where a.snpsht_genericTimePeriod = b.fcst_genericTimePeriod
        and a.snpsht_genericTimePeriod = date_trunc(current_date(), genericFrequency)
        and a.aggLevel = b.aggLevel;
    ''')

    with selectionOut:
        print(datetime.datetime.now().strftime('%H:%M:%S'), genericReplace('Reinforcement Learning selection output stored in `genericDataset.genericOutputTable`. Forecast process complete.',replaceDict))
except:
    with selectionOut:
        print(datetime.datetime.now().strftime('%H:%M:%S'), 'Error: Invalid input detected while using RL selection. Please verify inputs and correct pathing to the associated RL.js file and try again.')
